In [2]:
# ------------------------
# Imports and Utility Functions (as provided)
# ------------------------
from z3 import Int, Real, Bool, Solver, And, Or, Implies, If, Distinct, sat
from functools import reduce
from typing import List, Dict, Optional, Iterable, Any

# Global cache for Z3 variables
_z3_cache: Dict[str, Any] = {}

def _ensure_z3(expr: Any, default_type: str = 'int') -> Any:
    if isinstance(expr, str):
        if default_type == 'int':
            return create_int_var(expr)
        elif default_type == 'real':
            return create_real_var(expr)
        elif default_type == 'bool':
            return create_bool_var(expr)
        else:
            raise ValueError(f"Unknown default type: {default_type}")
    elif isinstance(expr, list):
        result = []
        for e in expr:
            result.append(_ensure_z3(e, default_type))
        return result
    else:
        return expr

def create_int_var(name: str) -> Any:
    if name in _z3_cache:
        return _z3_cache[name]
    var = Int(name)
    _z3_cache[name] = var
    return var

def create_real_var(name: str) -> Any:
    if name in _z3_cache:
        return _z3_cache[name]
    var = Real(name)
    _z3_cache[name] = var
    return var

def create_bool_var(name: str) -> Any:
    if name in _z3_cache:
        return _z3_cache[name]
    var = Bool(name)
    _z3_cache[name] = var
    return var

def equal_constraint(a: Any, b: Any) -> Any:
    a_cached = _ensure_z3(a, default_type='int')
    b_cached = _ensure_z3(b, default_type='int')
    return a_cached == b_cached

def not_equal_constraint(a: Any, b: Any) -> Any:
    a_cached = _ensure_z3(a, default_type='int')
    b_cached = _ensure_z3(b, default_type='int')
    return a_cached != b_cached

def less_than_constraint(a: Any, b: Any) -> Any:
    a_cached = _ensure_z3(a, default_type='int')
    b_cached = _ensure_z3(b, default_type='int')
    return a_cached < b_cached

def less_equal_constraint(a: Any, b: Any) -> Any:
    a_cached = _ensure_z3(a, default_type='int')
    b_cached = _ensure_z3(b, default_type='int')
    return a_cached <= b_cached

def greater_than_constraint(a: Any, b: Any) -> Any:
    a_cached = _ensure_z3(a, default_type='int')
    b_cached = _ensure_z3(b, default_type='int')
    return a_cached > b_cached

def greater_equal_constraint(a: Any, b: Any) -> Any:
    a_cached = _ensure_z3(a, default_type='int')
    b_cached = _ensure_z3(b, default_type='int')
    return a_cached >= b_cached

def sum_constraint(variables: List[Any], target_value: int) -> Any:
    z3_vars = []
    for var in variables:
        z3_vars.append(_ensure_z3(var, default_type='int'))
    addition = reduce(lambda a, b: a + b, z3_vars)
    return addition == target_value

def difference_constraint(a: Any, b: Any, target_value: int) -> Any:
    a_cached = _ensure_z3(a, default_type='int')
    b_cached = _ensure_z3(b, default_type='int')
    return a_cached - b_cached == target_value

def product_constraint(a: Any, b: Any, target_value: int) -> Any:
    a_cached = _ensure_z3(a, default_type='int')
    b_cached = _ensure_z3(b, default_type='int')
    return a_cached * b_cached == target_value

def division_constraint(a: Any, b: Any, target_value: float) -> Any:
    a_cached = _ensure_z3(a, default_type='int')
    b_cached = _ensure_z3(b, default_type='int')
    return a_cached / b_cached == target_value

def logical_and_constraint(constraints: List[Any]) -> Any:
    cached_list = []
    for c in constraints:
        if isinstance(c, str):
            cached_list.append(_ensure_z3(c, default_type='bool'))
        else:
            cached_list.append(c)
    return And(cached_list)

def logical_or_constraint(constraints: List[Any]) -> Any:
    cached_list = []
    for c in constraints:
        if isinstance(c, str):
            cached_list.append(_ensure_z3(c, default_type='bool'))
        else:
            cached_list.append(c)
    return Or(cached_list)

def implication_constraint(antecedent: Any, consequent: Any) -> Any:
    ant_cached = _ensure_z3(antecedent, default_type='bool')
    cons_cached = _ensure_z3(consequent, default_type='bool')
    return Implies(ant_cached, cons_cached)

def abs_constraint(expr: Any, expected: int) -> Any:
    expr_cached = _ensure_z3(expr, default_type='int')
    return If(expr_cached >= 0, expr_cached, -expr_cached) == expected

def in_domain_constraint(var: Any, domain: Iterable[int]) -> Any:
    var_cached = _ensure_z3(var, default_type='int')
    domain_constraints = []
    for value in domain:
        domain_constraints.append(var_cached == value)
    return Or(domain_constraints)

def all_different_constraint(variables: List[Any]) -> Any:
    vars_cached = []
    for v in variables:
        vars_cached.append(_ensure_z3(v, default_type='int'))
    return Distinct(vars_cached)

def solve_constraints(constraints: List[Any], timeout: Optional[int] = None) -> Optional[Dict[str, Any]]:
    z3_constraints = []
    for c in constraints:
        if isinstance(c, str):
            z3_constraints.append(_ensure_z3(c, default_type='bool'))
        else:
            z3_constraints.append(c)
    solver = Solver()
    if timeout is not None:
        solver.set("timeout", timeout)
    solver.add(z3_constraints)
    if solver.check() == sat:
        model = solver.model()
        result = {}
        for d in model.decls():
            result[str(d)] = model[d]
        return result
    else:
        return None

# ------------------------
# Puzzle Setup: Create Variables and Cache Them
# ------------------------

# Fliers (passengers)
brandi_var = create_int_var("brandi")
christie_var = create_int_var("christie")
eloise_var = create_int_var("eloise")
florence_var = create_int_var("florence")
jaime_var = create_int_var("jaime")

# Lucky Charms
coin_var = create_int_var("coin")
lucky_hat_var = create_int_var("lucky_hat")
shamrock_var = create_int_var("shamrock")
talisman_var = create_int_var("talisman")
wishbone_var = create_int_var("wishbone")

# ------------------------
# Domain Constraints for All Variables (Each must be in {1,2,3,4,5})
# ------------------------
dom_brandi = in_domain_constraint(brandi_var, [1, 2, 3, 4, 5])
dom_christie = in_domain_constraint(christie_var, [1, 2, 3, 4, 5])
dom_eloise = in_domain_constraint(eloise_var, [1, 2, 3, 4, 5])
dom_florence = in_domain_constraint(florence_var, [1, 2, 3, 4, 5])
dom_jaime = in_domain_constraint(jaime_var, [1, 2, 3, 4, 5])
dom_coin = in_domain_constraint(coin_var, [1, 2, 3, 4, 5])
dom_lucky_hat = in_domain_constraint(lucky_hat_var, [1, 2, 3, 4, 5])
dom_shamrock = in_domain_constraint(shamrock_var, [1, 2, 3, 4, 5])
dom_talisman = in_domain_constraint(talisman_var, [1, 2, 3, 4, 5])
dom_wishbone = in_domain_constraint(wishbone_var, [1, 2, 3, 4, 5])

# ------------------------
# All-Different Constraints for fliers and lucky charms
# ------------------------
all_diff_fliers = all_different_constraint([brandi_var, christie_var, eloise_var, florence_var, jaime_var])
all_diff_charms = all_different_constraint([coin_var, lucky_hat_var, shamrock_var, talisman_var, wishbone_var])

# ------------------------
# Encode the Clues
# ------------------------

# Clue 1:
# "The passenger leaving in February will bring their lucky hat." (February = 2)
clue1 = equal_constraint(lucky_hat_var, 2)

# Clue 2:
# "Christie will leave 1 month after Eloise." -> christie = eloise + 1
eloise_plus_1 = eloise_var + 1
clue2 = equal_constraint(christie_var, eloise_plus_1)

# Clue 3:
# "The passenger with the wishbone will leave 2 months after Eloise." -> wishbone = eloise + 2
eloise_plus_2 = eloise_var + 2
clue3 = equal_constraint(wishbone_var, eloise_plus_2)

# Clue 4:
# "The aerophobe leaving in May is either the aerophobe with the talisman or the aerophobe with the shamrock." (May = 5)
clue4a = equal_constraint(talisman_var, 5)
clue4b = equal_constraint(shamrock_var, 5)
clue4 = logical_or_constraint([clue4a, clue4b])

# Clue 5:
# "The flier with the coin will leave sometime after the passenger with the shamrock."
clue5 = less_than_constraint(shamrock_var, coin_var)

# Clue 6:
# "Of the flier with the coin and the passenger with the talisman, one is Brandi and the other will leave in May."
# Option 1: Brandi has the coin and the talisman is in May, and Brandi is not in May.
option1a = equal_constraint(coin_var, brandi_var)
option1b = equal_constraint(talisman_var, 5)
option1c = not_equal_constraint(brandi_var, 5)
option1 = logical_and_constraint([option1a, option1b, option1c])
# Option 2: Brandi has the talisman and the coin is in May, and Brandi is not in May.
option2a = equal_constraint(talisman_var, brandi_var)
option2b = equal_constraint(coin_var, 5)
option2c = not_equal_constraint(brandi_var, 5)
option2 = logical_and_constraint([option2a, option2b, option2c])
clue6 = logical_or_constraint([option1, option2])

# Clue 7:
# "Florence won't leave in March." (March = 3)
clue7 = not_equal_constraint(florence_var, 3)

# ------------------------
# Assemble All Constraints Sequentially
# ------------------------
constraints = []
constraints.append(dom_brandi)
constraints.append(dom_christie)
constraints.append(dom_eloise)
constraints.append(dom_florence)
constraints.append(dom_jaime)
constraints.append(dom_coin)
constraints.append(dom_lucky_hat)
constraints.append(dom_shamrock)
constraints.append(dom_talisman)
constraints.append(dom_wishbone)
constraints.append(all_diff_fliers)
constraints.append(all_diff_charms)
constraints.append(clue1)
constraints.append(clue2)
constraints.append(clue3)
constraints.append(clue4)
constraints.append(clue5)
constraints.append(clue6)
constraints.append(clue7)

# ------------------------
# Solve the Puzzle
# ------------------------
solution = solve_constraints(constraints)

# ------------------------
# Build the Final Mapping: Each Month (1=January, 2=February, etc.) -> (Flier, Lucky Charm)
# ------------------------
if solution:
    months = {1: "January", 2: "February", 3: "March", 4: "April", 5: "May"}
    month_to_flier = {}
    month_to_charm = {}

    # Process fliers: Cache each month
    brandi_month = solution["brandi"].as_long() if hasattr(solution["brandi"], "as_long") else int(solution["brandi"])
    christie_month = solution["christie"].as_long() if hasattr(solution["christie"], "as_long") else int(solution["christie"])
    eloise_month = solution["eloise"].as_long() if hasattr(solution["eloise"], "as_long") else int(solution["eloise"])
    florence_month = solution["florence"].as_long() if hasattr(solution["florence"], "as_long") else int(solution["florence"])
    jaime_month = solution["jaime"].as_long() if hasattr(solution["jaime"], "as_long") else int(solution["jaime"])

    month_to_flier[brandi_month] = "Brandi"
    month_to_flier[christie_month] = "Christie"
    month_to_flier[eloise_month] = "Eloise"
    month_to_flier[florence_month] = "Florence"
    month_to_flier[jaime_month] = "Jaime"

    # Process lucky charms: Cache each month
    coin_month = solution["coin"].as_long() if hasattr(solution["coin"], "as_long") else int(solution["coin"])
    lucky_hat_month = solution["lucky_hat"].as_long() if hasattr(solution["lucky_hat"], "as_long") else int(solution["lucky_hat"])
    shamrock_month = solution["shamrock"].as_long() if hasattr(solution["shamrock"], "as_long") else int(solution["shamrock"])
    talisman_month = solution["talisman"].as_long() if hasattr(solution["talisman"], "as_long") else int(solution["talisman"])
    wishbone_month = solution["wishbone"].as_long() if hasattr(solution["wishbone"], "as_long") else int(solution["wishbone"])

    month_to_charm[coin_month] = "Coin"
    month_to_charm[lucky_hat_month] = "Lucky Hat"
    month_to_charm[shamrock_month] = "Shamrock"
    month_to_charm[talisman_month] = "Talisman"
    month_to_charm[wishbone_month] = "Wishbone"

    # ------------------------
    # Print the Final Mapping
    # ------------------------
    print("Final Mapping:")
    for m in sorted(months.keys()):
        flier = month_to_flier.get(m, "Unknown")
        charm = month_to_charm.get(m, "Unknown")
        print(f"{months[m]}: Flier = {flier}, Lucky Charm = {charm}")
else:
    print("No solution found.")


Final Mapping:
January: Flier = Eloise, Lucky Charm = Shamrock
February: Flier = Christie, Lucky Charm = Lucky Hat
March: Flier = Jaime, Lucky Charm = Wishbone
April: Flier = Brandi, Lucky Charm = Coin
May: Flier = Florence, Lucky Charm = Talisman


In [5]:
from pprint import pprint
pprint(solution)

{'brandi': 4,
 'christie': 2,
 'coin': 4,
 'eloise': 1,
 'florence': 5,
 'jaime': 3,
 'lucky_hat': 2,
 'shamrock': 1,
 'talisman': 5,
 'wishbone': 3}


In [ ]:
{'Brandi': 3,
 'Christie': 2,
 'Eloise': 1,
 'Florence': 4,
 'Jaime.': 5,
 'May.': 1,
 'coin': 3,
 'lucky hat': 4,
 'shamrock': 2,
 'talisman': 1,
 'wishbone': 3,
 'wishbone.': 5}

In [8]:
a = {'April': 5,
 'Brandi': 3,
 'Christie': 2,
 'Eloise': 1,
 'February': 4,
 'Florence': 4,
 'Jaime.': 5,
 'January': 2,
 'March': 3,
 'May': 1,
 'May.': 1,
 'coin': 3,
 'lucky hat': 4,
 'shamrock': 2,
 'talisman': 1,
 'wishbone': 3,
 'wishbone.': 5}

In [10]:
pprint({k:v for k,v in a.items() if k not in ["January", "February", "March", "April", "May"]})

{'Brandi': 3,
 'Christie': 2,
 'Eloise': 1,
 'Florence': 4,
 'Jaime.': 5,
 'May.': 1,
 'coin': 3,
 'lucky hat': 4,
 'shamrock': 2,
 'talisman': 1,
 'wishbone': 3,
 'wishbone.': 5}


('January | Eloise | shamrock\n'
 'February | Christie | lucky hat\n'
 'March | Jaime | wishbone\n'
 'April | Brandi | coin\n'
 'May | Florence | talisman')

In [2]:
[
  {"function_name": "create_int_var", "parameters": ["January"]},
  {"function_name": "create_int_var", "parameters": ["February"]},
  {"function_name": "create_int_var", "parameters": ["March"]},
  {"function_name": "create_int_var", "parameters": ["April"]},
  {"function_name": "create_int_var", "parameters": ["May."]},
  {"function_name": "create_int_var", "parameters": ["Brandi"]},
  {"function_name": "create_int_var", "parameters": ["Christie"]},
  {"function_name": "create_int_var", "parameters": ["Eloise"]},
  {"function_name": "create_int_var", "parameters": ["Florence"]},
  {"function_name": "create_int_var", "parameters": ["Jaime."]},
  {"function_name": "create_int_var", "parameters": ["coin"]},
  {"function_name": "create_int_var", "parameters": ["lucky hat"]},
  {"function_name": "create_int_var", "parameters": ["shamrock"]},
  {"function_name": "create_int_var", "parameters": ["talisman"]},
  {"function_name": "create_int_var", "parameters": ["wishbone."]},
  {"function_name": "in_domain_constraint", "parameters": ["January"]},
  {"function_name": "in_domain_constraint", "parameters": ["February"]},
  {"function_name": "in_domain_constraint", "parameters": ["March"]},
  {"function_name": "in_domain_constraint", "parameters": ["April"]},
  {"function_name": "in_domain_constraint", "parameters": ["May."]},
  {"function_name": "in_domain_constraint", "parameters": ["Brandi"]},
  {"function_name": "in_domain_constraint", "parameters": ["Christie"]},
  {"function_name": "in_domain_constraint", "parameters": ["Eloise"]},
  {"function_name": "in_domain_constraint", "parameters": ["Florence"]},
  {"function_name": "in_domain_constraint", "parameters": ["Jaime."]},
  {"function_name": "in_domain_constraint", "parameters": ["coin"]},
  {"function_name": "in_domain_constraint", "parameters": ["lucky hat"]},
  {"function_name": "in_domain_constraint", "parameters": ["shamrock"]},
  {"function_name": "in_domain_constraint", "parameters": ["talisman"]},
  {"function_name": "in_domain_constraint", "parameters": ["wishbone."]},
  {"function_name": "all_different_constraint", "parameters": ["months"]},
  {"function_name": "all_different_constraint", "parameters": ["fliers"]},
  {"function_name": "all_different_constraint", "parameters": ["lucky charms"]},
  {"function_name": "equal_constraint", "parameters": ["February", "lucky hat"]},
  {"function_name": "difference_constraint", "parameters": ["Christie", "Eloise",    
  "target_value: 1"]},
  {"function_name": "difference_constraint", "parameters": ["wishbone.", "Eloise", "target_value: 2"]},
  {"function_name": "logical_or_constraint", "parameters": [
  "constraints: ['equal_constraint(May., talisman)', 'equal_constraint(May., shamrock)']"]},
  {"function_name": "greater_than_constraint", "parameters": ["coin", "shamrock"]},
  {"function_name": "logical_or_constraint", "parameters": [
  "constraints: ['logical_and_constraint(constraints=[\"equal_constraint(coin, Brandi)\", \"equal_constraint(May., talisman)\"] )\", 'logical_and_constraint(constraints=[\"equal_constraint(coin, May.)\", \"equal_constraint(Brandi, talisman)\"] )\"]"]},
  {"function_name": "not_equal_constraint", "parameters": ["Florence", "March"]}
]

[{'function_name': 'create_int_var', 'parameters': ['January']},
 {'function_name': 'create_int_var', 'parameters': ['February']},
 {'function_name': 'create_int_var', 'parameters': ['March']},
 {'function_name': 'create_int_var', 'parameters': ['April']},
 {'function_name': 'create_int_var', 'parameters': ['May.']},
 {'function_name': 'create_int_var', 'parameters': ['Brandi']},
 {'function_name': 'create_int_var', 'parameters': ['Christie']},
 {'function_name': 'create_int_var', 'parameters': ['Eloise']},
 {'function_name': 'create_int_var', 'parameters': ['Florence']},
 {'function_name': 'create_int_var', 'parameters': ['Jaime.']},
 {'function_name': 'create_int_var', 'parameters': ['coin']},
 {'function_name': 'create_int_var', 'parameters': ['lucky hat']},
 {'function_name': 'create_int_var', 'parameters': ['shamrock']},
 {'function_name': 'create_int_var', 'parameters': ['talisman']},
 {'function_name': 'create_int_var', 'parameters': ['wishbone.']},
 {'function_name': 'in_domain

In [ ]:
df["answer"].iloc[58]

In [1]:
from high_level_wrapper_tools import *


In [2]:
categories = {'months': ['January', 'February', 'March', 'April', 'May.'],
 'fliers': ['Brandi', 'Christie', 'Eloise', 'Florence', 'Jaime.'],
 'lucky charms': ['coin', 'lucky hat', 'shamrock', 'talisman', 'wishbone.']}

In [3]:
global categories

In [4]:
category = "months"
define_variables(category, type="int", categories=categories)

{'months': [[January, February, March, April, May.]]}

In [ ]:
from z3 import (
    Int, Real, Bool, Solver, And, Or, Implies, If, Distinct, sat,
    ExprRef, BoolRef
)
from typing import List, Dict, Optional, Any

class ConstraintSolver:
    """
    A class that provides a high-level interface for creating and solving
    constraints using the Z3 solver. All constraint functions accept only
    string identifiers that map to unique Z3 variable objects stored in the variables_cache.
    """

    def __init__(self, categories: Dict[str, List[str]]):
        """
        Initialize the constraint solver with categories.
        
        Args:
            categories (Dict[str, List[str]]): A dictionary mapping category names
                                               to lists of variable names (strings).
        """
        self.categories = categories
        # Centralized cache: keys are variable names, values are Z3 variable objects.
        self.variables_cache: Dict[str, ExprRef] = {}

    def ensure_z3(self, var_name: str, default_type: str = 'int') -> ExprRef:
        """
        Given a variable name, return its corresponding Z3 variable from the variables_cache.
        If the variable does not exist, create it with the given type, store it, and then return it.
        
        Args:
            var_name (str): The name of the variable.
            default_type (str): The type to use if a new variable must be created ('int', 'real', or 'bool').
            
        Returns:
            ExprRef: The Z3 variable associated with var_name.
        """
        if var_name in self.variables_cache:
            return self.variables_cache[var_name]
        else:
            if default_type == 'int':
                var = Int(var_name)
            elif default_type == 'real':
                var = Real(var_name)
            elif default_type == 'bool':
                var = Bool(var_name)
            else:
                raise ValueError(f"Unknown default type: {default_type}")
            self.variables_cache[var_name] = var
            return var

    def define_constraint_variables(self, category: str, type_: str = "int") -> Dict[str, List[ExprRef]]:
        """
        For a given category, convert every variable name (a string) to its corresponding
        Z3 object (creating it if necessary) and return a mapping from the category to the list
        of these Z3 variables.
        
        Args:
            category (str): The key to look up variable names in self.categories.
            type_ (str): The type for creating Z3 variables ('int', 'real', or 'bool').
            
        Returns:
            Dict[str, List[ExprRef]]: A dictionary mapping the category to its list of Z3 variables.
        """
        if category not in self.categories:
            raise ValueError(f"Category '{category}' not found in categories.")
        
        cat_vars = [self.ensure_z3(name, default_type=type_) for name in self.categories[category]]
        return {category: cat_vars}

    def in_domain_constraint(self, category: str) -> List[BoolRef]:
        """
        For a given category, create a domain constraint for each variable in that category.
        The domain is defined as the set {1, 2, ..., N} where N is the number of items in
        the category as determined by the categories dict.
        
        Args:
            category (str): The category name.
            
        Returns:
            List[BoolRef]: A list of domain constraints, one per variable in the category.
        """
        if category not in self.categories:
            raise ValueError(f"Category '{category}' not found in categories.")
        
        domain_size = len(self.categories[category])
        constraints = []
        for var_name in self.categories[category]:
            var = self.ensure_z3(var_name, default_type='int')
            # Create constraint: var == 1 OR var == 2 OR ... OR var == domain_size
            constraints.append(Or([var == i for i in range(1, domain_size + 1)]))
        return constraints

    def get_domain_constraints(self, category: str) -> List[BoolRef]:
        """
        Alias for in_domain_constraint.
        
        Args:
            category (str): The category name.
            
        Returns:
            List[BoolRef]: A list of domain constraints for all variables in the category.
        """
        return self.in_domain_constraint(category)

    def all_different_constraint(self, category: str) -> BoolRef:
        """
        Create a constraint that ensures all variables in the given category have distinct values.
        
        Args:
            category (str): The category name.
            
        Returns:
            BoolRef: A constraint asserting that every variable in the category is distinct.
        """
        if category not in self.categories:
            raise ValueError(f"Category '{category}' not found in categories.")
        vars_list = [self.ensure_z3(name, default_type='int') for name in self.categories[category]]
        return Distinct(vars_list)

    def equal_constraint(self, a: str, b: str) -> BoolRef:
        """
        Create an equality constraint between two variables.
        
        Args:
            a (str): The name of the first variable.
            b (str): The name of the second variable.
            
        Returns:
            BoolRef: The constraint (var_a == var_b).
        """
        return self.ensure_z3(a, default_type='int') == self.ensure_z3(b, default_type='int')

    def not_equal_constraint(self, a: str, b: str) -> BoolRef:
        """
        Create a non-equality constraint between two variables.
        
        Args:
            a (str): The name of the first variable.
            b (str): The name of the second variable.
            
        Returns:
            BoolRef: The constraint (var_a != var_b).
        """
        return self.ensure_z3(a, default_type='int') != self.ensure_z3(b, default_type='int')

    def less_than_constraint(self, a: str, b: str) -> BoolRef:
        """
        Create a constraint asserting that one variable is less than another.
        
        Args:
            a (str): The name of the first variable.
            b (str): The name of the second variable.
            
        Returns:
            BoolRef: The constraint (var_a < var_b).
        """
        return self.ensure_z3(a, default_type='int') < self.ensure_z3(b, default_type='int')

    def less_equal_constraint(self, a: str, b: str) -> BoolRef:
        """
        Create a constraint asserting that one variable is less than or equal to another.
        
        Args:
            a (str): The name of the first variable.
            b (str): The name of the second variable.
            
        Returns:
            BoolRef: The constraint (var_a <= var_b).
        """
        return self.ensure_z3(a, default_type='int') <= self.ensure_z3(b, default_type='int')

    def greater_than_constraint(self, a: str, b: str) -> BoolRef:
        """
        Create a constraint asserting that one variable is greater than another.
        
        Args:
            a (str): The name of the first variable.
            b (str): The name of the second variable.
            
        Returns:
            BoolRef: The constraint (var_a > var_b).
        """
        return self.ensure_z3(a, default_type='int') > self.ensure_z3(b, default_type='int')

    def greater_equal_constraint(self, a: str, b: str) -> BoolRef:
        """
        Create a constraint asserting that one variable is greater than or equal to another.
        
        Args:
            a (str): The name of the first variable.
            b (str): The name of the second variable.
            
        Returns:
            BoolRef: The constraint (var_a >= var_b).
        """
        return self.ensure_z3(a, default_type='int') >= self.ensure_z3(b, default_type='int')

    def sum_constraint(self, var_names: List[str], target_value: int) -> BoolRef:
        """
        Create a constraint that the sum of the variables (by name) equals target_value.
        
        Args:
            var_names (List[str]): A list of variable names.
            target_value (int): The target sum.
            
        Returns:
            BoolRef: The constraint (sum(vars) == target_value).
        """
        z3_vars = [self.ensure_z3(name, default_type='int') for name in var_names]
        return sum(z3_vars) == target_value

    def difference_constraint(self, a: str, b: str, target_value: int) -> BoolRef:
        """
        Create a constraint that the difference of two variables equals target_value.
        
        Args:
            a (str): The name of the first variable.
            b (str): The name of the second variable.
            target_value (int): The target difference.
            
        Returns:
            BoolRef: The constraint (var_a - var_b == target_value).
        """
        return self.ensure_z3(a, default_type='int') - self.ensure_z3(b, default_type='int') == target_value

    def product_constraint(self, a: str, b: str, target_value: int) -> BoolRef:
        """
        Create a constraint that the product of two variables equals target_value.
        
        Args:
            a (str): The name of the first variable.
            b (str): The name of the second variable.
            target_value (int): The target product.
            
        Returns:
            BoolRef: The constraint (var_a * var_b == target_value).
        """
        return self.ensure_z3(a, default_type='int') * self.ensure_z3(b, default_type='int') == target_value

    def division_constraint(self, a: str, b: str, target_value: float) -> BoolRef:
        """
        Create a constraint that the division of two variables equals target_value.
        Assumes the denominator is non-zero.
        
        Args:
            a (str): The name of the numerator.
            b (str): The name of the denominator.
            target_value (float): The target quotient.
            
        Returns:
            BoolRef: The constraint (var_a / var_b == target_value).
        """
        return self.ensure_z3(a, default_type='int') / self.ensure_z3(b, default_type='int') == target_value

    def abs_constraint(self, var_name: str, expected: int) -> BoolRef:
        """
        Create a constraint asserting that the absolute value of a variable equals expected.
        
        Args:
            var_name (str): The name of the variable.
            expected (int): The expected absolute value.
            
        Returns:
            BoolRef: The constraint (If(var >= 0, var, -var) == expected).
        """
        var = self.ensure_z3(var_name, default_type='int')
        return If(var >= 0, var, -var) == expected

    def implication_constraint(self, antecedent: str, consequent: str) -> BoolRef:
        """
        Create an implication constraint between two boolean variables.
        
        Args:
            antecedent (str): The name of the antecedent variable.
            consequent (str): The name of the consequent variable.
            
        Returns:
            BoolRef: The constraint (Implies(var_ant, var_cons)).
        """
        ant = self.ensure_z3(antecedent, default_type='bool')
        cons = self.ensure_z3(consequent, default_type='bool')
        return Implies(ant, cons)

    
    def logical_and_constraint(self, constraints: List[BoolRef]) -> BoolRef:
        """
        Create a compound constraint that is the logical AND of the provided constraints.
        
        Args:
            constraints (List[BoolRef]): A list of Z3 boolean constraints.
            
        Returns:
            BoolRef: A constraint representing the conjunction (AND) of all constraints.
        """
        return And(constraints)

    def logical_or_constraint(self, constraints: List[BoolRef]) -> BoolRef:
        """
        Create a compound constraint that is the logical OR of the provided constraints.
        
        Args:
            constraints (List[BoolRef]): A list of Z3 boolean constraints.
            
        Returns:
            BoolRef: A constraint representing the disjunction (OR) of all constraints.
        """
        return Or(constraints)

    def solve(self, constraints: List[BoolRef]) -> Optional[Dict[str, Any]]:
        """
        Solve the given set of constraints using Z3 and return a mapping of variable names to values.
        
        Args:
            constraints (List[BoolRef]): A list of Z3 constraints.
            
        Returns:
            Optional[Dict[str, Any]]: A dictionary mapping variable names to their solution values if satisfiable;
                                      otherwise, None.
        """
        solver = Solver()
        for c in constraints:
            solver.add(c)
        if solver.check() == sat:
            model = solver.model()
            return {name: model[var] for name, var in self.variables_cache.items() if var in model}
        return None


In [ ]:
# def logical_and_constraint(self, var_names: List[str]) -> BoolRef:
    #     """
    #     Create a logical AND constraint over a list of boolean variable names.
        
    #     Args:
    #         var_names (List[str]): A list of boolean variable names.
            
    #     Returns:
    #         BoolRef: The constraint (And(var1, var2, ...)).
    #     """
    #     bool_vars = [self.ensure_z3(name, default_type='bool') for name in var_names]
    #     return And(bool_vars)

    # def logical_or_constraint(self, var_names: List[str]) -> BoolRef:
    #     """
    #     Create a logical OR constraint over a list of boolean variable names.
        
    #     Args:
    #         var_names (List[str]): A list of boolean variable names.
            
    #     Returns:
    #         BoolRef: The constraint (Or(var1, var2, ...)).
    #     """
    #     bool_vars = [self.ensure_z3(name, default_type='bool') for name in var_names]
    #     return Or(bool_vars)

In [ ]:
# def process_for_logical_operation(cs, info):

#     constraints = []
#     var_a, var_b, operator, value = [""] * 4
    
#     function_name = info["function_name"]
#     parameters = info["parameters"]

#     try:
#         for param in parameters:
            
#             params =  param.split("|")
            
#             if len(params) == 3:
#                 var_a , var_b, operator = params
            
#             elif len(params) == 4:
#                 var_a, var_b, operator, value = params

            
#             if "==" in operator:
#                 res = cs.equal_constraint(var_a, var_b)

#             elif ">=" in operator:
#                 res = cs.greater_equal_constraint(var_a,var_b)
            
#             elif "<=" in operator:
#                 res = cs.less_equal_constraint(var_a,var_b)

#             elif ">" in operator:
#                 res = cs.greater_than_constraint(var_a,var_b)
            
#             elif "<" in operator:
#                 res = cs.less_than_constraint(var_a,var_b)
            
#             elif "!=" in operator:
#                 res = cs.not_equal_constrain(var_a, var_b)
            
#             constraints.append(res)
#     except Exception as e:
#         print(e, type(var_a), type(var_b), type(operator))

#     return constraints

# #

In [13]:
from z3 import Solver, Int, Distinct, Or, And, Not


class LogicPuzzleSolver:
    """
    Generic logic grid puzzle solver using Z3.

    Each item in each category is represented by an integer Z3 variable, constrained to a
    common domain [1..N], where N is the size of the largest category. Matching two items
    means equating their variables; uniqueness within a category is enforced via all-different.

    Usage:
        categories = {
            "Person": ["Alice", "Bob", "Charlie"],
            "City":   ["Paris", "London", "Tokyo"],
            "Fruit":  ["Apple", "Banana", "Cherry"]
        }
        solver = LogicPuzzleSolver(categories)
        solver.set_all_different("Person")
        solver.set_equal("Person", "Alice", "City", "Paris")
        solver.set_not_equal("Person", "Bob", "Fruit", "Apple")
        solver.set_less_than("City", "Paris", "City", "Tokyo")
        result = solver.solve()
        print(result)
    """

    def __init__(self, categories: dict[str, list[str]]):
        """
        Initialize solver with categories and create Z3 Int variables for each item.

        Args:
            categories: Mapping from category name to list of item names.
        """
        self.solver = Solver()
        self.categories = categories
        # Determine maximum category size for domain bounds
        self.domain_size = max(len(items) for items in categories.values())
        # Lookup: { category: { item_name: z3.Int } }
        self.vars: dict[str, dict[str, Int]] = {}

        # Create variables and domain constraints
        for cat, items in categories.items():
            self.vars[cat] = {}
            for item in items:
                v = Int(f"{cat}_{item}")
                self.vars[cat][item] = v
                # Domain: 1 <= v <= domain_size
                self.solver.add(v >= 1, v <= self.domain_size)

    def set_all_different(self, category: str) -> None:
        """
        Constrain all items in 'category' to have distinct values.

        Args:
            category: Name of the category to apply all-different.
        """
        vars_list = list(self.vars[category].values())
        self.solver.add(Distinct(*vars_list))

    def set_value(self, category: str, item: str, value: int) -> None:
        """
        Fix a specific item to a given integer value.

        Args:
            category: Category name.
            item:     Item name within the category.
            value:    Integer value to assign (within domain).
        """
        self.solver.add(self.vars[category][item] == value)

    def set_equal(self, cat1: str, item1: str, cat2: str, item2: str) -> None:
        """
        Constrain two items (possibly from different categories) to match.

        Args:
            cat1, item1: First category and item.
            cat2, item2: Second category and item.
        """
        v1 = self.vars[cat1][item1]
        v2 = self.vars[cat2][item2]
        self.solver.add(v1 == v2)

    def set_not_equal(self, cat1: str, item1: str, cat2: str, item2: str) -> None:
        """
        Ensure two items are not matched (values differ).

        Args:
            cat1, item1: First category and item.
            cat2, item2: Second category and item.
        """
        v1 = self.vars[cat1][item1]
        v2 = self.vars[cat2][item2]
        self.solver.add(v1 != v2)

    def set_less_than(self, cat1: str, item1: str, cat2: str, item2: str) -> None:
        """
        Constrain the value of (cat1,item1) to be less than (cat2,item2).

        Args:
            cat1,item1: Item that comes first (smaller value).
            cat2,item2: Item that comes later (larger value).
        """
        v1 = self.vars[cat1][item1]
        v2 = self.vars[cat2][item2]
        self.solver.add(v1 < v2)

    def set_greater_than(self, cat1: str, item1: str, cat2: str, item2: str) -> None:
        """
        Constrain the value of (cat1,item1) to be greater than (cat2,item2).

        Args:
            cat1,item1: Item that comes later (larger value).
            cat2,item2: Item that comes first (smaller value).
        """
        v1 = self.vars[cat1][item1]
        v2 = self.vars[cat2][item2]
        self.solver.add(v1 > v2)

    def set_item_before(self, category: str, item1: str, item2: str) -> None:
        """
        Within the same category, constrain item1 to precede item2.

        Args:
            category: Category name.
            item1:    Item that should come before.
            item2:    Item that should come after.
        """
        self.set_less_than(category, item1, category, item2)

    def set_item_after(self, category: str, item1: str, item2: str) -> None:
        """
        Within the same category, constrain item1 to follow item2.

        Args:
            category: Category name.
            item1:    Item that should come after.
            item2:    Item that should come before.
        """
        self.set_greater_than(category, item1, category, item2)

    def set_item_not_in(self, category: str, item: str, disallowed_value: int) -> None:
        """
        Forbid a specific integer value for an item.

        Args:
            category:         Category name.
            item:             Item name.
            disallowed_value: Integer value not allowed for this item.
        """
        v = self.vars[category][item]
        self.solver.add(v != disallowed_value)

    def set_exact_difference(
        self,
        cat1: str, item1: str,
        cat2: str, item2: str,
        diff: int
    ) -> None:
        """
        Enforce that the absolute difference between two items equals diff.

        Args:
            cat1,item1: First category/item.
            cat2,item2: Second category/item.
            diff:       Required absolute difference.
        """
        v1 = self.vars[cat1][item1]
        v2 = self.vars[cat2][item2]
        self.solver.add(Or(v1 - v2 == diff, v2 - v1 == diff))

    def set_either_or_value(
        self,
        category: str,
        item: str,
        val1: int,
        val2: int
    ) -> None:
        """
        Restrict an item to be exactly one of two integer values.

        Args:
            category: Category name.
            item:     Item name.
            val1:     First allowed value.
            val2:     Second allowed value.
        """
        v = self.vars[category][item]
        self.solver.add(Or(v == val1, v == val2))

    def set_either_or_match(
        self,
        cat1: str, item1: str,
        cat2: str, item2: str,
        cat3: str, item3: str,
        cat4: str, item4: str
    ) -> None:
        """
        Encode an exclusive either/or clue involving two pairings.

        Either (cat1:item1) matches (cat2:item2) OR (cat3:item3) matches (cat4:item4),
        but not both.

        Args:
            cat1,item1: First pairing's category and item.
            cat2,item2: First pairing's matched category and item.
            cat3,item3: Second pairing's category and item.
            cat4,item4: Second pairing's matched category and item.
        """
        v1 = self.vars[cat1][item1]
        v2 = self.vars[cat2][item2]
        v3 = self.vars[cat3][item3]
        v4 = self.vars[cat4][item4]
        cond1 = (v1 == v2)
        cond2 = (v3 == v4)
        # Exclusive OR: one or the other, not both
        self.solver.add(Or(cond1, cond2), Not(And(cond1, cond2)))

    def set_one_of(
        self,
        category: str,
        item: str,
        allowed_values: list[int]
    ) -> None:
        """
        Constrain an item to be one of a list of integer values.

        Args:
            category:        Category name.
            item:            Item name.
            allowed_values:  List of permitted integer values.
        """
        v = self.vars[category][item]
        ors = [v == val for val in allowed_values]
        self.solver.add(Or(*ors))

    def solve(self) -> dict[str, dict[str, int]] | None:
        """
        Solve the accumulated constraints and return assignments.

        Returns:
            A nested dict mapping category->item->solved integer, or None if unsat.
        """
        if self.solver.check().r != 1:  # 1 means sat in Z3
            return None
        model = self.solver.model()
        result: dict[str, dict[str, int]] = {}
        for cat, items in self.vars.items():
            result[cat] = {}
            for item, var in items.items():
                result[cat][item] = model[var].as_long()
        return result
    
    def format_solution_table(self, solution: Dict[str, Dict[str, int]]) -> str:
        """
        Format a solved solution as a human-readable table.

        Args:
            solution (Dict[str, Dict[str, int]]): The solution mapping from solve().

        Returns:
            str: Text table with columns for each category.
        """
        cats = list(self.categories.keys())
        first = cats[0]
        rows: List[List[str]] = []
        for itm in self.categories[first]:
            pos = solution[first][itm]
            row = [itm] + [next((x for x, v in solution[c].items() if v == pos), '') for c in cats[1:]]
            rows.append(row)

        widths = [max(len(c), max(len(r[i]) for r in rows)) for i, c in enumerate(cats)]
        header = ' | '.join(c.ljust(widths[i]) for i, c in enumerate(cats))
        sep = '-+-'.join('-' * widths[i] for i in range(len(cats)))
        lines = [header, sep] + [' | '.join(r[i].ljust(widths[i]) for i in range(len(cats))) for r in rows]
        return '\n'.join(lines)


In [ ]:
Generic Z3 Logic Grid Puzzle Solver Class

Logic grid puzzles (also known as Zebra or Einstein puzzles) involve matching items across multiple categories based on a series of logical clues. Each category contains a list of unique items, and the puzzle’s solution is an assignment that links each item in one category to exactly one item in every other category. For example, if categories are "Person", "City", and "Fruit", a valid solution might pair each person with one unique city and one unique fruit. Solving these puzzles with a constraint solver like Z3 involves encoding each clue as a constraint and ensuring the assignments are consistent and unique​
wdj-consulting.com
​
wdj-consulting.com
.
Class Design Overview

We can design a reusable Python class (e.g., LogicPuzzleSolver) to handle the setup and solving of these puzzles using the Z3 SMT solver. The class will be initialized with a dictionary of categories, where each key is a category name and the value is a list of items in that category. For instance:

categories = {
    "Person": ["Alice", "Bob", "Charlie"],
    "City":   ["Paris", "London", "Tokyo"],
    "Fruit":  ["Apple", "Banana", "Cherry"]
}
solver = LogicPuzzleSolver(categories)

Internal Representation: During initialization, the class will create a Z3 integer variable for each item in each category. These integer variables serve as placeholders for the position or index of that item in the solution. If each category has N items, we can constrain these integers to the range 1 through N (or 0 through N-1) to represent a unique index for each item​
wdj-consulting.com
. All items in a category will thus map to an integer domain of size N. By giving each item a number, matching two items (from different categories) is as simple as equating their numbers in the Z3 solver.

    Each Z3 int variable is stored in a lookup dictionary for easy reference. For example, self.vars["Person"]["Alice"] might be a Z3 Int for Alice’s position.

    The class will automatically add basic domain constraints, e.g. if "Person" has 3 items, each Person variable is constrained 1 <= var <= 3. This prevents Z3 from assigning out-of-range values (which, by default, an unconstrained Int could be any integer​
    wdj-consulting.com
    ).

Uniqueness Constraints: In logic puzzles, it’s implied that each item in a category matches a different item in another category (one-to-one matching). To enforce this, we often need an all-different constraint per category, ensuring no two items share the same index. This is analogous to saying "each category’s items occupy unique positions" – equivalent to the puzzle rule that, e.g., no two people share the same city​
wdj-consulting.com
. We will provide a helper method to add this constraint for a given category (using Z3’s Distinct or pairwise inequalities under the hood).
Constraint Helper Methods

The solver class provides a variety of atomic, sequential methods to add common puzzle constraints. Each method adds one self-contained constraint to the Z3 solver. The methods take only simple types (string names for categories/items or integers for numeric values) and internally translate them to the corresponding Z3 variables via the lookup dictionary. There is no method nesting or chaining required – each call independently adds a constraint, and the user typically calls multiple methods in sequence to encode all clues. (Each method returns None or self – we do not produce complex return values that could be nested; all the “action” happens internally by adding to the Z3 solver.)

Below is a summary of common helper methods and what they enforce. Each method would have a clear docstring in the class definition explaining its usage and effect:

    set_all_different(category: str) – Ensures all items in the given category have distinct values. This adds a constraint that the Z3 variables for every pair of items in category are not equal (often implemented with Distinct(...) on the list of variables). This models the rule that each item in a category must map to a unique position (no two people have the same city, etc.)​
    wdj-consulting.com
    . Docstring example: “Constrain all items in category to have different numerical assignments (no two items share the same index).”

    set_value(category: str, item: str, value: int) – Assigns a specific value to a given item. This is typically used to hard-code a clue like “Alice is in position 3” or “The event happened in March (assuming March corresponds to 3)”. Internally does solver.add(var(category,item) == value). Docstring: “Fixes the value of category:item to the given integer value (e.g., setting an item’s position or index).”

    set_equal(category1: str, item1: str, category2: str, item2: str) – Declares that two items are the same entity or matched. In terms of indices, it means item1 of category1 and item2 of category2 share the exact same number. For example, set_equal("Person","Alice", "City","Paris") means Alice’s index equals Paris’s index, so Alice is paired with Paris. Docstring: “Constrain category1:item1 to be matched with category2:item2. Internally sets their variables equal, meaning they occupy the same position in the solution.”

    set_not_equal(category1: str, item1: str, category2: str, item2: str) – Specifies that two items are different and cannot be matched. For example, if a clue says “Alice did not go to Tokyo,” you’d call set_not_equal("Person","Alice", "City","Tokyo"). Internally adds a constraint that the two variables are not equal. Docstring: “Ensure that category1:item1 and category2:item2 do not share the same value (they cannot be matched together).”

    set_less_than(category1: str, item1: str, category2: str, item2: str) – Imposes an ordering constraint: the value of item1 is less than the value of item2. This is useful when one category (or both) represents an ordered attribute like time, rank, or numeric measures. For instance, if item values correspond to days or positions, this could encode “Alice arrived before Bob” or “Alice’s number is lower than Bob’s.” Internally: solver.add(var1 < var2). Docstring: “Add a numeric constraint that category1:item1’s value is less than category2:item2’s value.”

    set_greater_than(category1: str, item1: str, category2: str, item2: str) – The inverse of the above, ensures item1’s value is greater than item2’s value. (Semantically, set_greater_than(A,x, B,y) is equivalent to set_less_than(B,y, A,x)). Docstring: “Constrain category1:item1 to have a value greater than category2:item2.”

    set_item_before(category: str, item1: str, item2: str) – A convenience for ordering within the same category. It ensures item1 comes before item2 in that category’s order (i.e., value(item1) < value(item2)). For example, set_item_before("Month", "March", "July") if the Month category’s values correspond to calendar order. Internally identical to a less-than constraint on the same category. Docstring: “Ensure that within category X, the item item1 is in an earlier position (lower number) than item2.”

    set_item_after(category: str, item1: str, item2: str) – The counterpart to set_item_before, asserting item1 comes after (has a higher number than) item2 in the same category. Docstring: “Ensure that item1 appears later (greater index) than item2 in category X.”

    set_item_not_in(category: str, item: str, disallowed_value: int) – Forbids a specific value for an item. If item values have intrinsic meaning (e.g., a numeric value like an age or a position number), this ensures the item’s number is not the given value. For example, “Alice did not finish in 1st place” would be set_item_not_in("Person","Alice", 1). Internally: solver.add(var(item) != disallowed_value). Docstring: “Prohibit category:item from taking the value disallowed_value.”

    set_exact_difference(category1: str, item1: str, category2: str, item2: str, diff: int) – Enforces that the difference between two items’ values is exactly diff. This can model clues like “The London trip took 2 days longer than the Paris trip” or “Alice is 3 years older than Bob”. If diff is positive, the constraint is often interpreted as an absolute difference: one value is exactly diff greater than the other. We can implement this by adding an Or-condition: either var1 - var2 == diff or var2 - var1 == diff, unless the clue implies a specific direction. For a non-directional clue (just "diff apart"), we ensure the absolute difference equals diff. Internally: solver.add(Or(var1 - var2 == d, var2 - var1 == d)). Docstring: “Ensure that the values of category1:item1 and category2:item2 differ by exactly diff (in either order).”

    set_either_or_value(category: str, item: str, val1: int, val2: int) – Represents a clue where an item’s value is one of two possibilities. For example, “The concert was either 3 or 5 hours long.” This adds a disjunctive constraint that item’s value is val1 or val2. Internally it uses a logical OR: solver.add(Or(var == val1, var == val2))​
    ericpony.github.io
    . Because a single item’s value can’t be both at once, this effectively means either one or the other. Docstring: “Restrict category:item to take either value val1 or val2 (one of the two, no other value allowed).”

    set_either_or_match(category1: str, item1: str, category2: str, item2: str, category3: str, item3: str, category4: str, item4: str) – Encodes an “either-or” clue involving two possible matching scenarios. For instance, “Either Alice is matched with Paris, or Bob is matched with Tokyo.” This would be set_either_or_match("Person","Alice","City","Paris", "Person","Bob","City","Tokyo"). The method adds a constraint that at least one of those two matchings holds true. Internally: solver.add(Or(var(Alice)==var(Paris), var(Bob)==var(Tokyo))). This means the solver can satisfy the clue by making one of the pairings valid. In logic puzzles, either-or clues are usually exclusive (only one of the two is true, not both)​
    reddit.com
    . If needed, the user could add additional constraints to prevent both conditions from happening simultaneously (e.g., ensure the two conditions can’t be true together). Docstring: “Encode an either/or clue: either (category1:item1 matches category2:item2) or (category3:item3 matches category4:item4). At least one of these pairings will hold in the solution.”

    set_one_of(category: str, item: str, list_of_values: list[int]) – Constrains an item’s value to be one among a list of allowed values. This is a generalization of the either/or value case. For example, if a clue said “Charlie lives on either the 1st, 3rd, or 5th floor”, one could call set_one_of("Person","Charlie", [1,3,5]). Internally it adds Or(var==v1, var==v2, ..., var==vn) ensuring the item’s value is in that set. Docstring: “Limit category:item to one of the values in the provided list (any of those values is acceptable, others are not).”

Each of these helper methods is self-contained – it converts the provided arguments to the corresponding Z3 variables and immediately adds the appropriate constraint to the solver. There is no complex return value or chaining; the user simply calls these methods one after another to incrementally build the full puzzle constraint model. This design makes the API easy to use and less error-prone: every call corresponds directly to one clue or rule.


You can use the above text for you research analysis. I need a generic class to solve logic grid puzzle problems by mapping them to constrained satisfaction problem. The input would be cateogries and items in the form of dictionary and based on some clues we have to relate one item from a category to one item from other categories. So to achieve this task I want you to create a class that tries to implement constraints using z-solver python binding. Each function should essentially add a constraint to the solver and it should be simple in terms of definition and parameter requirements. I dont admit lengthy input parameters. No function should be implemented in such a way that nesting can happen. That is calling a function inside a function. Only sequential calling is allowed.   Make as many generic functions as you want because I would be testing it on problems and it should solve most of them accurately. You would be penalized if your implementation is wrong. Once the solver has solved the problem, nicely populate it in a table depicting the relation. Also the first category always decides the order of mapping. So you can only rewire the item mapping from other categories apart from the first one. I hope you understand the requirments clearly. Give me the best possible python class that is generic to solve most logic problems. Remember only sequential functions calls are allowed when solving a problem. Have this in mind. And also do not have more than 3 input parameters for any function. It becomes cumbersome. 

In [27]:
from z3 import Solver, Int, Distinct, Or, Not, Abs

class LogicPuzzleSolver:
    def __init__(self, categories: dict):
        # Store categories (maintain insertion order of keys as given)
        self.categories = categories
        self.category_names = list(categories.keys())
        if not self.category_names:
            raise ValueError("No categories provided")
        self.first_category = self.category_names[0]
        self.num_positions = len(categories[self.first_category])
        # Ensure all categories have the same number of items for one-to-one mapping
        for cat, items in categories.items():
            if len(items) != self.num_positions:
                raise ValueError(f"Category '{cat}' length {len(items)} != {self.num_positions} of first category")
        # Initialize Z3 solver
        self.solver = Solver()
        # Dictionary to hold Z3 Int vars for each item
        self.item_vars = {}
        # Create a Z3 Int variable for each item in each category
        for cat, items in categories.items():
            for item in items:
                var = Int(f"{cat}_{item}")
                # Constrain domain to 1..N (inclusive)
                self.solver.add(var >= 1, var <= self.num_positions)
                self.item_vars[item] = var
        # Add all-different constraints for items within each category
        for cat, items in categories.items():
            vars_list = [self.item_vars[item] for item in items]
            # All items in this category must have distinct positions
            self.solver.add(Distinct(*vars_list))
    
    def add_all_different(self, items: list):
        """Ensure all given items are in distinct positions."""
        vars_list = [self.item_vars[item] for item in items]
        self.solver.add(Distinct(*vars_list))
    
    def add_equal(self, item1: str, item2: str):
        """Constrain item1 and item2 to be at the same position."""
        self.solver.add(self.item_vars[item1] == self.item_vars[item2])
    
    def add_not_equal(self, item1: str, item2: str):
        """Constrain item1 and item2 to be at different positions."""
        self.solver.add(self.item_vars[item1] != self.item_vars[item2])
    
    def add_before(self, item1: str, item2: str):
        """Constrain item1 to come before item2 (i.e., lower index)."""
        self.solver.add(self.item_vars[item1] < self.item_vars[item2])
    
    def add_after(self, item1: str, item2: str):
        """Constrain item1 to come after item2 (i.e., higher index)."""
        self.solver.add(self.item_vars[item1] > self.item_vars[item2])
    
    def assign_position(self, item: str, position: int):
        """Assign a specific position index to the item."""
        self.solver.add(self.item_vars[item] == position)
    
    def add_difference(self, item1: str, item2: str, diff: int):
        """Constrain the absolute difference in positions of item1 and item2."""
        # Ensure diff is non-negative
        if diff < 0:
            raise ValueError("Difference must be non-negative")
        # Use Z3's Abs (absolute value) to enforce |pos1 - pos2| = diff
        self.solver.add(Abs(self.item_vars[item1] - self.item_vars[item2]) == diff)
    
    def add_either_or(self, item: str, option1: str, option2: str):
        """Constrain that item is matched with either option1 or option2."""
        self.solver.add(Or(self.item_vars[item] == self.item_vars[option1],
                           self.item_vars[item] == self.item_vars[option2]))
    
    def add_in(self, item: str, options: list):
        """Constrain that item is matched with one of the items in the options list."""
        # Build a chain of OR conditions for equality with any option
        or_conditions = [self.item_vars[item] == self.item_vars[opt] for opt in options]
        self.solver.add(Or(*or_conditions))
    
    def add_not_in(self, item: str, options: list):
        """Constrain that item is NOT matched with any of the items in the options list."""
        # Or of equalities for all options, then negate to ensure none match
        or_conditions = [self.item_vars[item] == self.item_vars[opt] for opt in options]
        self.solver.add(Not(Or(*or_conditions)))
    
    def solve(self):
        """Solve the puzzle and return a formatted solution table (or None if unsolvable)."""
        if self.solver.check() != sat:
            print("No solution exists or constraints are unsatisfiable.")
            return None
        model = self.solver.model()
        # Prepare header row
        headers = list(self.category_names)
        # Build a mapping from category and position -> item for easy lookup
        category_at_pos = {cat: {} for cat in self.category_names}
        for cat, items in self.categories.items():
            for item in items:
                pos_val = model[self.item_vars[item]].as_long()
                category_at_pos[cat][pos_val] = item
        # Create table rows ordered by the first category's item list order
        output_lines = []
        output_lines.append(" | ".join(headers))
        output_lines.append("-" * (len(headers) * 3 + 3 * (len(headers)-1)))  # separator line
        first_cat_items = self.categories[self.first_category]
        for item in first_cat_items:
            pos = model[self.item_vars[item]].as_long()
            row = [item]  # start with the item from first category
            # for each other category, find the item at the same position
            for cat in self.category_names[1:]:
                mapped_item = category_at_pos[cat].get(pos, "")
                row.append(mapped_item)
            output_lines.append(" | ".join(row))
        result_table = "\n".join(output_lines)
        print(result_table)
        return result_table


In [ ]:
from z3 import Solver, Int, Distinct, Or, Not, Abs, sat

class LogicPuzzleSolver:
    """
    Generic Z3-based logic grid puzzle solver.

    Attributes:
        categories (dict[str, list[str]]): category name → list of item names
        item_vars (dict[str, Int]): item name → Z3 Int variable
        solver (Solver): Z3 solver instance
        z3_cache (dict): cache for compound Z3 expressions
    """

    def __init__(self, categories: dict):
        """
        Initialize with one Int var per item, domain 1..N, and all-different per category.
        The first category's items are anchored to positions 1..N in the given order.

        Args:
            categories (dict[str, list[str]]): mapping from category to items. All categories must
                have the same number of items.
        Raises:
            ValueError: if no categories provided or category lengths differ.
        """
        self.categories = categories
        self.category_names = list(categories.keys())
        if not self.category_names:
            raise ValueError("No categories provided")

        self.first_category = self.category_names[0]
        n = len(categories[self.first_category])
        for cat, items in categories.items():
            if len(items) != n:
                raise ValueError(f"Category '{cat}' length {len(items)} != {n}")

        self.solver = Solver()
        self.item_vars = {}
        self.z3_cache = {}

        # Create Int var for each item and constrain 1..N
        for items in categories.values():
            for it in items:
                v = Int(it)
                self.solver.add(v >= 1, v <= n)
                self.item_vars[it] = v

        # All-different within each category
        for items in categories.values():
            vars_list = [self.item_vars[it] for it in items]
            self.solver.add(Distinct(*vars_list))

        # Anchor first category: fix positions 1..N in given order
        for idx, it in enumerate(self.categories[self.first_category], start=1):
            self.solver.add(self.item_vars[it] == idx)

    def add_all_different(self, items: list):
        """Ensure all given items occupy distinct positions."""
        self.solver.add(Distinct(*[self.item_vars[it] for it in items]))

    def add_equal(self, a: str, b: str):
        """Enforce item a and item b share the same position."""
        self.solver.add(self.item_vars[a] == self.item_vars[b])

    def add_not_equal(self, a: str, b: str):
        """Enforce item a and item b occupy different positions."""
        self.solver.add(self.item_vars[a] != self.item_vars[b])

    def add_before(self, a: str, b: str):
        """Enforce a's position < b's position."""
        self.solver.add(self.item_vars[a] < self.item_vars[b])

    def add_after(self, a: str, b: str):
        """Enforce a's position > b's position."""
        self.solver.add(self.item_vars[a] > self.item_vars[b])

    def assign_position(self, item: str, pos: int):
        """Fix item to a specific numeric position."""
        self.solver.add(self.item_vars[item] == pos)

    def add_difference(self, a: str, b: str, diff: int):
        """Enforce |pos(a) - pos(b)| == diff."""
        if diff < 0:
            raise ValueError("Difference must be non-negative")
        key = ('diff', a, b, diff)
        if key not in self.z3_cache:
            expr = Abs(self.item_vars[a] - self.item_vars[b]) == diff
            self.z3_cache[key] = expr
        self.solver.add(self.z3_cache[key])

    def add_either_or(self, item: str, opt1: str, opt2: str):
        """Enforce item matches either opt1 or opt2."""
        self.solver.add(Or(
            self.item_vars[item] == self.item_vars[opt1],
            self.item_vars[item] == self.item_vars[opt2]
        ))

    def add_in(self, item: str, options: list):
        """Enforce item matches one in options list."""
        self.solver.add(Or(*[self.item_vars[item] == self.item_vars[o] for o in options]))

    def add_not_in(self, item: str, options: list):
        """Enforce item matches none in options list."""
        self.solver.add(Not(Or(*[self.item_vars[item] == self.item_vars[o] for o in options])))

    def add_cross_not_equal(self, cat1: str, cat2: str):
        """
        For every item in cat1 and every item in cat2,
        enforce they cannot share the same position.
        """
        for a in self.categories[cat1]:
            for b in self.categories[cat2]:
                self.solver.add(self.item_vars[a] != self.item_vars[b])

    def add_permutation(self, cat1: str, cat2: str, perm_name: str):
        """
        Enforce that items of cat1 map to items of cat2
        according to perm_name: 'identity', 'reverse', or 'rotate_k'.
        """
        src = self.categories[cat2]
        if perm_name == 'identity':
            permuted = list(src)
        elif perm_name == 'reverse':
            permuted = list(reversed(src))
        elif perm_name.startswith('rotate_'):
            k = int(perm_name.split('_', 1)[1]) % len(src)
            permuted = src[k:] + src[:k]
        else:
            raise ValueError(f"Unknown perm_name '{perm_name}'")
        for a, b in zip(self.categories[cat1], permuted):
            self.solver.add(self.item_vars[a] == self.item_vars[b])

    def solve(self):
        """Solve and return a formatted solution table, or None if unsatisfiable."""
        if self.solver.check() != sat:
            print("No solution exists or constraints are unsatisfiable.")
            return None

        model = self.solver.model()
        # Build category->pos->item mapping
        cat_at = {cat: {} for cat in self.category_names}
        for cat, items in self.categories.items():
            for it in items:
                pos = model[self.item_vars[it]].as_long()
                cat_at[cat][pos] = it

        # Build textual table
        headers = " | ".join(self.category_names)
        sep = "-" * len(headers)
        lines = [headers, sep]
        for it in self.categories[self.first_category]:
            pos = model[self.item_vars[it]].as_long()
            row = [it] + [cat_at[c].get(pos, "") for c in self.category_names[1:]]
            lines.append(" | ".join(row))

        result = "\n".join(lines)
        print(result)
        return result


In [40]:

categories = {
    "months":    ["January", "February", "March",   "April"],
    "diplomats": ["Jacobson","Lee",      "Pickett","Willis"],
    "durations": ["3 day",   "5 day",    "7 day",  "10 day"],
    "capitals":  ["Dublin",  "Oslo",     "Tirane", "Warsaw"]
}
solver = LogicPuzzleSolver(categories)

# 1. Warsaw before Lee
solver.add_before("Warsaw", "Lee")

# 2. Neither 7 day nor March is Pickett
solver.add_not_equal("7 day", "Pickett")
solver.add_not_equal("March",  "Pickett")

# 3. Of January and the 3 day visit, one is Willis and the other is Oslo
solver.add_either_or("Willis", "January", "3 day")
solver.add_either_or("Oslo",    "January", "3 day")

# 4. Warsaw before 10 day
solver.add_before("Warsaw", "10 day")

# 5. Neither 7 day nor Jacobson goes to Oslo
solver.add_not_equal("7 day",   "Oslo")
solver.add_not_equal("Jacobson","Oslo")

# 6. February = Warsaw
solver.add_equal("February", "Warsaw")

# 7. Willis is either at Warsaw or at March
solver.add_either_or("Willis", "Warsaw", "March")

# 8. Jacobson is either at Tirane or at January
solver.add_either_or("Jacobson", "Tirane", "January")

# Solve and display the ordered table
res_ = solver.solve()


months | diplomats | durations | capitals
-----------------------------------------
January | Pickett | 5 day | Oslo
February | Willis | 3 day | Warsaw
March | Jacobson | 7 day | Tirane
April | Lee | 10 day | Dublin


In [41]:
# Initialize the solver with our categories
puzzle = LogicPuzzleSolver({
    'prices': ['$400', '$425', '$450', '$475', '$500'],
    'attendees': ['Jack Jordan', 'Lily Little', 'Mitch Mayo', 'Ned Norris', 'Ora Osborne'],
    'gifts': ['blender', 'blu-ray player', 'juicer', 'laptop', 'planter'],
    'towns': ['Clovis', 'Decatur City', 'Eustis', 'Fruitland', 'Sun City']
})

# Add all the clear constraints
puzzle.add_difference('Jack Jordan', 'planter', 25)
puzzle.add_equal('$400', 'Eustis')
puzzle.add_not_equal('$425', 'blender')
puzzle.add_not_equal('Jack Jordan', 'Decatur City')
puzzle.add_difference('Lily Little', 'juicer', 25)
puzzle.add_not_equal('Ora Osborne', 'Clovis')

# Now for clue 4, I'll try the only possible option that doesn't contradict clue 2
# Since $400 is Eustis (clue 2), it can't also be Clovis
# So $500 must be Clovis and $400 must be planter
puzzle.add_equal('$500', 'Clovis')
puzzle.add_equal('$400', 'planter')

# Solve the puzzle
solution = puzzle.solve()

No solution exists or constraints are unsatisfiable.


In [ ]:
No solution exists or constraints are unsatisfiable.

In [ ]:


Gold

('January | Pickett | 5 day | Oslo\n'
 'February | Willis | 3 day | Warsaw\n'
 'March | Jacobson | 10 day | Tirane\n'
 'April | Lee | 7 day | Dublin')

'January | Pickett | 5 day | Oslo\nFebruary | Willis | 3 day | Warsaw\nMarch | Jacobson | 10 day | Tirane\nApril | Lee | 7 day | Dublin'

In [37]:
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[17], line 122
    119 solver.set_either_or_match("diplomats", "Jacobson", "capitals", "Tirane", "diplomats", "Jacobson", "months", "January")
    121 result = solver.solve("months")
--> 122 table = solver.format_solution_table(result)
    123 print(table)

Cell In[17], line 86
     84 rows: List[List[str]] = []
     85 for itm in self.categories[first]:
---> 86     pos = solution[first][itm]
     87     row = [itm] + [next((x for x, v in solution[c].items() if v == pos), '') for c in cats[1:]]
     88     rows.append(row)

TypeError: list indices must be integers or slices, not str

SyntaxError: invalid syntax (119944893.py, line 1)

In [ ]:
from z3 import Solver, Bool, Or, And, Xor, Not, Implies, If, Sum

class GenericLogicPuzzleSolver:
    def __init__(self, categories):
        """
        Initialize the solver with categories and their items.
        :param categories: Ordered dict or list of (category_name, [items]) pairs.
                            The first category in this sequence is the anchor.
        """
        # Store categories and items
        if isinstance(categories, dict):
            # If categories provided as dict
            self.categories = list(categories.keys())
            self.items = {cat: list(categories[cat]) for cat in categories}
        else:
            # If provided as list of tuples
            self.categories = [cat for cat, _ in categories]
            self.items = {cat: list(items) for cat, items in categories}
        self.first_cat = self.categories[0]  # first category name
        self.solver = Solver()
        # Internal lookup for direct Bool variables (first category vs others)
        self.var = {}  # self.var[category][(first_item, category_item)] -> Bool
        # Cache for composite conditions (relations not directly first-vs-other)
        self.relation_cache = {}
        # Cache for numeric values of category items (for sum/difference constraints)
        self.numeric_val = {}
        # Determine numeric categories: if all items are numbers, map them
        for cat, items in self.items.items():
            vals = []
            for it in items:
                try:
                    vals.append(int(it))
                except:
                    try:
                        vals.append(float(it))
                    except:
                        vals = None
                        break
            if vals is not None:
                # If successfully parsed all items as numbers, store mapping
                self.numeric_val[cat] = {item: vals[i] for i, item in enumerate(items)}
        # Create Z3 Bool variables for each pairing between first category and others
        for cat in self.categories:
            if cat == self.first_cat:
                continue
            self.var[cat] = {}
            for f_item in self.items[self.first_cat]:
                for item in self.items[cat]:
                    # Naming the variable for clarity (optional)
                    var_name = f"{f_item}_{cat}_{item}"
                    self.var[cat][(f_item, item)] = Bool(var_name)
        # Add base constraints: each first-category item matches one item in each other category, and vice versa
        # For each non-first category:
        for cat in self.categories:
            if cat == self.first_cat:
                continue
            # 1. Each first_cat item gets exactly one item from this category
            for f_item in self.items[self.first_cat]:
                # Exactly one of {f_item is paired with each item in cat}
                conds = [self.var[cat][(f_item, item)] for item in self.items[cat]]
                # Each first item must have at least one match in this category
                self.solver.add(Or(*conds))
                # And at most one (pairwise mutual exclusion or sum == 1)
                # Using sum == 1 for exactly one true:
                self.solver.add(Sum([If(c, 1, 0) for c in conds]) == 1)
            # 2. Each item in this category is taken by exactly one first_cat item
            for item in self.items[cat]:
                conds = [self.var[cat][(f_item, item)] for f_item in self.items[self.first_cat]]
                # At least one first_cat item has this 'item'
                self.solver.add(Or(*conds))
                # At most one (exactly one)
                self.solver.add(Sum([If(c, 1, 0) for c in conds]) == 1)
        # Note: Between non-first categories themselves, consistency is naturally enforced by these constraints.
    def _get_condition_expr(self, cat1, item1, cat2, item2):
        """
        Internal helper to get the Z3 Boolean expression representing the condition 
        that item1 of category cat1 is matched with item2 of category cat2.
        """
        # If the condition involves the first category directly:
        if cat1 == self.first_cat and cat2 in self.var:
            return self.var[cat2][(item1, item2)]
        if cat2 == self.first_cat and cat1 in self.var:
            return self.var[cat1][(item2, item1)]
        # Otherwise, neither cat1 nor cat2 is the first category (both are non-first categories).
        # We need to express that some first_cat item links them.
        # Create a key with categories in a fixed order to avoid duplicates in cache
        key = (cat1, item1, cat2, item2) if (cat1, cat2) == (cat1, cat2) or cat1 < cat2 else (cat2, item2, cat1, item1)
        if key in self.relation_cache:
            return self.relation_cache[key]
        # Build the expression: there exists a first_cat item F such that F-cat1=item1 AND F-cat2=item2
        # We express existence as a disjunction over all first_cat items.
        conds = []
        for f_item in self.items[self.first_cat]:
            expr1 = self._get_condition_expr(cat1, item1, self.first_cat, f_item) if cat1 != self.first_cat else (self.var[cat2][(f_item, item1)] if False else False)
            expr2 = self._get_condition_expr(cat2, item2, self.first_cat, f_item) if cat2 != self.first_cat else (self.var[cat1][(f_item, item2)] if False else False)
            # The above lines need to get the bools for "f_item has item1 in cat1" and "f_item has item2 in cat2".
            # If cat1 is non-first, we have var[cat1][(f_item, item1)].
            # If cat1 happened to be first, expr1 is trivial True for the chosen f_item being itself, but cat1 can't be first here because we handle that above.
            if cat1 != self.first_cat and cat2 != self.first_cat:
                expr1 = self.var[cat1][(f_item, item1)]
                expr2 = self.var[cat2][(f_item, item2)]
            conds.append(And(expr1, expr2))
        relation_expr = Or(*conds)
        self.relation_cache[key] = relation_expr
        return relation_expr
    def implies(self, premise, conclusion):
        """
        Add an implication constraint: if 'premise' then 'conclusion'.
        Each of premise and conclusion is a tuple (Category, Item, Category2, Item2) representing an assignment condition.
        """
        expr_p = self._condition_tuple_to_expr(premise)
        expr_q = self._condition_tuple_to_expr(conclusion)
        self.solver.add(Implies(expr_p, expr_q))
    def either_or(self, cond1, cond2):
        """
        Add an inclusive either-or constraint: at least one of cond1 or cond2 is true.
        cond1 and cond2 are tuples representing assignment conditions.
        """
        expr1 = self._condition_tuple_to_expr(cond1)
        expr2 = self._condition_tuple_to_expr(cond2)
        self.solver.add(Or(expr1, expr2))
    def xor(self, cond1, cond2):
        """
        Add an exclusive OR constraint: exactly one of cond1 or cond2 is true (not both).
        """
        expr1 = self._condition_tuple_to_expr(cond1)
        expr2 = self._condition_tuple_to_expr(cond2)
        # Z3's Xor returns True if an odd number of arguments are True, so for two args it's exactly one.
        self.solver.add(Xor(expr1, expr2))
    def one_of(self, *conds):
        """
        Add a one-of constraint for multiple conditions: exactly one of the given conditions is true.
        Accepts conditions as multiple tuple arguments or a single list/tuple of condition tuples.
        """
        # Support passing a list of conds or varargs
        if len(conds) == 1 and isinstance(conds[0], (list, tuple)) and not isinstance(conds[0][0], str):
            cond_list = conds[0]
        else:
            cond_list = conds
        exprs = [self._condition_tuple_to_expr(c) for c in cond_list]
        # At least one is true:
        self.solver.add(Or(*exprs))
        # At most one is true:
        # Pairwise mutual exclusion or use sum == 1
        self.solver.add(Sum([If(e, 1, 0) for e in exprs]) == 1)
    def at_least_one(self, *conds):
        """
        Require that at least one of the given conditions is true.
        """
        if len(conds) == 1 and isinstance(conds[0], (list, tuple)) and not isinstance(conds[0][0], str):
            cond_list = conds[0]
        else:
            cond_list = conds
        exprs = [self._condition_tuple_to_expr(c) for c in cond_list]
        self.solver.add(Or(*exprs))
    def not_all(self, *conds):
        """
        Require that not all of the given conditions are true simultaneously.
        (At least one must be false.)
        """
        if len(conds) == 1 and isinstance(conds[0], (list, tuple)) and not isinstance(conds[0][0], str):
            cond_list = conds[0]
        else:
            cond_list = conds
        exprs = [self._condition_tuple_to_expr(c) for c in cond_list]
        self.solver.add(Not(And(*exprs)))
    def mutually_exclusive(self, cond1, cond2):
        """
        Ensure two conditions cannot both be true at the same time.
        (Logical NOT of their conjunction.)
        """
        expr1 = self._condition_tuple_to_expr(cond1)
        expr2 = self._condition_tuple_to_expr(cond2)
        self.solver.add(Not(And(expr1, expr2)))
    def sum_equal(self, category, first_items, total_value):
        """
        For a numeric category, constrain the sum of the values of the given first-category items to equal total_value.
        :param category: The name of the numeric category (must have numeric item values).
        :param first_items: List of items from the first category whose values to sum.
        :param total_value: The target sum (int or float).
        """
        assert category in self.numeric_val, f"Category {category} must be numeric to use sum constraint."
        # Build sum of values for each specified first_item
        sum_expr = 0
        for f_item in first_items:
            # Each f_item has exactly one item in 'category'; sum up its numeric value
            # We create an expression for the value of f_item in this category
            # Cache each first_item's value expression for this category to reuse if needed
            cache_key = (category, f_item)
            if cache_key in self.relation_cache:
                # Reuse cached Int expression if exists
                value_expr = self.relation_cache[cache_key]
            else:
                # Build value expression: sum of (value * Bool) for each possible item
                terms = []
                for item, val in self.numeric_val[category].items():
                    # Bool var for (f_item is assigned 'item' in category)
                    bool_var = self._condition_tuple_to_expr((self.first_cat, f_item, category, item))
                    # term = value * If(bool_var, 1, 0)  (If returns that value if true, 0 if false)
                    terms.append(If(bool_var, val, 0))
                value_expr = Sum(*terms)
                self.relation_cache[cache_key] = value_expr
            sum_expr = sum_expr + value_expr
        # Constrain the total sum
        self.solver.add(sum_expr == total_value)
    def _condition_tuple_to_expr(self, condition):
        """
        Convert a user-provided condition tuple into a Z3 Boolean expression via internal lookup.
        Accepts tuples of form (Cat1, Item1, Cat2, Item2).
        """
        cat1, item1, cat2, item2 = condition
        return self._get_condition_expr(cat1, item1, cat2, item2)
    def solve(self):
        """
        Solve the puzzle. Returns a dictionary mapping each first-category item to its matched items in other categories.
        If multiple solutions exist, returns one solution (not necessarily the only one).
        """
        if self.solver.check() != sat:
            return None  # No solution found
        model = self.solver.model()
        solution = {}  # {first_item: {category: item}}
        for f_item in self.items[self.first_cat]:
            solution[f_item] = {}
        # For each non-first category, find which item is true for each first_item
        for cat in self.categories:
            if cat == self.first_cat:
                continue
            for f_item in self.items[self.first_cat]:
                for item in self.items[cat]:
                    if model.evaluate(self.var[cat][(f_item, item)]):
                        solution[f_item][cat] = item
                        break
        return solution
    def solution_table(self):
        """
        Solve and return the solution in a formatted table string (first category as rows).
        """
        sol = self.solve()
        if sol is None:
            return "No solution found."
        # Prepare table header
        headers = [self.first_cat] + [cat for cat in self.categories if cat != self.first_cat]
        # Determine column widths
        col_widths = {}
        for h in headers:
            col_widths[h] = max(len(h), max((len(sol[f][h]) if h in sol[f] else 0) for f in sol) if h != self.first_cat else len(max(sol.keys(), key=len)))
        # Build header row
        header_row = " | ".join(h.ljust(col_widths[h]) for h in headers)
        line = "-+-".join("-"*col_widths[h] for h in headers)
        # Build data rows
        rows = []
        for f_item, assigns in sol.items():
            row = f_item.ljust(col_widths[self.first_cat])
            for cat in headers[1:]:
                row += " | " + assigns.get(cat, "").ljust(col_widths[cat])
            rows.append(row)
        # Combine all parts
        table = header_row + "\n" + line + "\n" + "\n".join(rows)
        return table


In [ ]:
Okay, let's modify the Z3ConstraintBuilder class to incorporate category-level operations and the requested tabular model output.

Changes:

__init__: Added self.category_vars to track which variables belong to which category.

New Methods:

declare_category_variables: Creates variables for all items in a category.

add_category_domain_constraint: Applies a simple domain constraint (e.g., var > val) to all variables of a category.

add_category_distinct_constraint: Adds a Distinct constraint for all variables of a category.

get_model Modification: Overhauled to identify category variables, group them, and format the output into tables using pipe symbols (|).

Variable Naming Convention: Category-based variables will be named CategoryName_ItemName (e.g., Price_Apple, Price_Orange). _get_z3_obj is not modified to automatically parse this, relying on the user referencing them by this full name if needed outside the category methods.

# Install z3-solver: pip install z3-solver
import z3
import re # For basic literal parsing
from collections import defaultdict

class Z3ConstraintBuilder:
    """
    A wrapper class for the z3-solver library to build constraints
    using methods that accept only string inputs (max 3 per method).

    Manages Z3 variables, constants, sorts, and the solver instance
    using a provided cache dictionary. Uses a categories dictionary
    to define potential EnumSorts and groups for bulk operations.

    Includes category-level variable declaration and constraints,
    and formatted table output for models involving categories.
    """

    def __init__(self, categories: dict, cache: dict):
        """
        Initializes the Z3ConstraintBuilder.

        Args:
            categories (dict): A dictionary where keys are category names (str)
                               and values are lists of item names (str).
                               Example: {'Products': ['Apple', 'Banana'], 'Colors': ['Red', 'Green']}
            cache (dict): A dictionary used to store Z3 objects (Solver,
                          variables, constants, sorts) keyed by string names.
                          This dictionary should be persistent if you want to
                          reuse declared items across multiple builder instances
                          sharing the same logical state. It *must* be provided.
                          Example: {} (for a new problem)
        """
        if not isinstance(categories, dict):
            raise TypeError("`categories` must be a dictionary.")
        if not isinstance(cache, dict):
            raise TypeError("`cache` must be a dictionary.")

        self.categories = categories
        self.cache = cache
        # Stores mapping: category_name -> [var_name1, var_name2, ...]
        # Useful for applying category-wide constraints and formatting output
        self.category_vars = defaultdict(list)
        # Populate category_vars if cache already contains category variables
        self._repopulate_category_vars_from_cache()


        # Ensure solver instance exists in cache, create if not
        if 'solver' not in self.cache:
            self.cache['solver'] = z3.Solver()
        elif not isinstance(self.cache['solver'], z3.Solver):
            raise TypeError("Cache item 'solver' is not a z3.Solver instance.")

        self._solver = self.cache['solver'] # Convenience accessor

        # Pre-cache basic boolean literals for convenience
        if 'True' not in self.cache:
            self.cache['True'] = z3.BoolVal(True)
        if 'False' not in self.cache:
            self.cache['False'] = z3.BoolVal(False)

    def _repopulate_category_vars_from_cache(self):
        """Internal: Scans cache to rebuild self.category_vars mapping."""
        self.category_vars.clear()
        for var_name in self.cache.keys():
            if '_' in var_name:
                parts = var_name.split('_', 1)
                category_guess = parts[0]
                item_guess = parts[1]
                # Check if category_guess is a valid category AND item_guess is in its items
                if category_guess in self.categories and item_guess in self.categories[category_guess]:
                     # Check if the cached object is a Z3 variable type we expect
                     obj = self.cache[var_name]
                     if z3.is_expr(obj) and obj.decl().kind() == z3.Z3_OP_UNINTERPRETED:
                         self.category_vars[category_guess].append(var_name)

    # --- Internal Helper ---
    # ( _get_z3_obj remains largely the same as before)
    def _get_z3_obj(self, name: str):
        """
        Retrieves or creates a Z3 object based on the input string.
        Order of lookup:
        1. Cached object (variable, constant, enum value, category_item vars)
        2. Boolean literals ('True', 'False')
        3. Integer literal
        4. Real literal (containing '.')
        5. Enum constant based on category definitions (must be declared first)
        Raises ValueError if the string cannot be resolved.
        """
        if not isinstance(name, str):
             raise TypeError(f"Input must be a string, got {type(name)}: {name}")

        # 1. Check cache (covers declared vars, category_item vars, enum consts, True/False)
        if name in self.cache:
            return self.cache[name]

        # 2. Boolean literals (already checked via cache)

        # 3. Integer Literal
        if re.fullmatch(r"-?\d+", name):
            try:
                val = int(name)
                return z3.IntVal(val)
            except ValueError:
                pass

        # 4. Real Literal
        if '.' in name or 'e' in name.lower():
             if re.fullmatch(r"-?(\d+(\.\d*)?|\.\d+)([eE][-+]?\d+)?", name):
                try:
                    return z3.RealVal(name)
                except ValueError:
                   raise ValueError(f"Could not parse '{name}' as a Real literal.")

        # 5. Enum Constant Check (must be declared via declare_enum_sort)
        # If 'Red' is passed and Colors enum was declared, 'Red' should be in cache.

        raise ValueError(
            f"Could not resolve string '{name}' to a known variable (incl. category vars like Cat_Item), "
            f"literal (Int, Real, Bool), or declared Enum constant. "
            f"Ensure variables/enums are declared first."
        )

    # --- Variable Declaration Methods ---

    def declare_variable(self, name: str, type_str: str, param1: str = None):
        """ Declares a single Z3 variable. (Mostly unchanged) """
        if name in self.cache:
            raise ValueError(f"Variable or object named '{name}' already exists in cache.")
        # Prevent declaring single vars that look like category vars if category exists
        if '_' in name:
             parts = name.split('_', 1)
             if parts[0] in self.categories and parts[1] in self.categories[parts[0]]:
                 raise ValueError(f"Variable name '{name}' follows the 'Category_Item' pattern. "
                                  f"Use 'declare_category_variables' for category '{parts[0]}' instead.")

        z3_var = None
        type_str_lower = type_str.lower()

        try:
            if type_str_lower == 'int':
                z3_var = z3.Int(name)
            elif type_str_lower == 'real':
                z3_var = z3.Real(name)
            elif type_str_lower == 'bool':
                z3_var = z3.Bool(name)
            elif type_str_lower == 'bitvec':
                if param1 is None: raise ValueError("BitVec type requires size in param1.")
                size = int(param1)
                if size <= 0: raise ValueError("BitVec size must be positive.")
                z3_var = z3.BitVec(name, size)
            elif type_str_lower == 'array':
                 raise ValueError("Use 'declare_array_variable' for Array type.")
            elif type_str_lower == 'enum':
                 raise ValueError("Use 'declare_enum_variable' for Enum type.")
            else:
                # Try resolving type_str as a declared EnumSort name
                sort_cache_key = f"sort_{type_str}"
                if sort_cache_key in self.cache:
                    enum_sort = self.cache[sort_cache_key]
                    if z3.is_sort(enum_sort) and z3.is_datatype(enum_sort):
                        z3_var = z3.Const(name, enum_sort) # Var of enum type
                    else:
                        raise ValueError(f"Cache key '{sort_cache_key}' exists but is not an EnumSort.")
                else:
                     raise ValueError(f"Unsupported or unknown variable type_str: '{type_str}'.")

        except (ValueError, TypeError) as e:
            raise ValueError(f"Error declaring variable '{name}' of type '{type_str}': {e}")

        self.cache[name] = z3_var
        print(f"Declared {type_str} variable '{name}'" + (f" with param '{param1}'" if param1 else ""))


    # --- Category-Level Variable Declaration ---
    def declare_category_variables(self, category_name: str, type_str: str, param1: str = None):
        """
        Declares variables for all items within a specified category.
        Variables are named 'CategoryName_ItemName'.

        Args:
            category_name (str): The key in the `categories` dictionary.
            type_str (str): The type for these variables ('Int', 'Real', 'Bool',
                            'BitVec', or a declared Enum category name).
            param1 (str, optional): Required for 'BitVec' (size).

        Raises:
            ValueError: If category not found, type invalid, or items already declared.
        """
        if category_name not in self.categories:
            raise ValueError(f"Category '{category_name}' not found in categories dictionary.")
        if not self.categories[category_name]:
             print(f"Warning: Category '{category_name}' is empty. No variables declared.")
             return

        items = self.categories[category_name]
        print(f"Declaring variables for category '{category_name}' (items: {items}) with type '{type_str}'...")

        created_vars = []
        try:
            sort_obj = None # For Enum types
            type_str_lower = type_str.lower()

            # Pre-determine sort or type parameters
            if type_str_lower == 'bitvec':
                if param1 is None: raise ValueError("BitVec type requires size in param1.")
                bv_size = int(param1)
                if bv_size <= 0: raise ValueError("BitVec size must be positive.")
            elif type_str_lower not in ['int', 'real', 'bool']:
                # Assume type_str is an Enum category name
                sort_cache_key = f"sort_{type_str}"
                if sort_cache_key not in self.cache:
                     raise ValueError(f"EnumSort '{type_str}' has not been declared. Call declare_enum_sort first.")
                sort_obj = self.cache[sort_cache_key]
                if not (z3.is_sort(sort_obj) and z3.is_datatype(sort_obj)):
                    raise ValueError(f"'{type_str}' resolved to something other than an EnumSort.")


            for item in items:
                var_name = f"{category_name}_{item}"
                if var_name in self.cache:
                    raise ValueError(f"Variable '{var_name}' already exists in cache.")

                z3_var = None
                if type_str_lower == 'int':
                    z3_var = z3.Int(var_name)
                elif type_str_lower == 'real':
                    z3_var = z3.Real(var_name)
                elif type_str_lower == 'bool':
                    z3_var = z3.Bool(var_name)
                elif type_str_lower == 'bitvec':
                    z3_var = z3.BitVec(var_name, bv_size)
                elif sort_obj: # If it was resolved as an Enum sort
                     z3_var = z3.Const(var_name, sort_obj)
                else:
                     # This case should have been caught earlier
                     raise ValueError(f"Internal error: Unhandled type '{type_str}'")

                self.cache[var_name] = z3_var
                self.category_vars[category_name].append(var_name) # Track it
                created_vars.append(var_name)

            print(f"Successfully declared {len(created_vars)} variables for category '{category_name}'.")

        except (ValueError, TypeError) as e:
             # Clean up partially created vars from cache and category_vars if error occurs
             for var_name in created_vars:
                 if var_name in self.cache: del self.cache[var_name]
                 if category_name in self.category_vars and var_name in self.category_vars[category_name]:
                     self.category_vars[category_name].remove(var_name)
             if not self.category_vars[category_name]:
                 del self.category_vars[category_name]
             raise ValueError(f"Error declaring category variables for '{category_name}': {e}")


    # --- Other Declaration Methods (Array, EnumSort, EnumVar) ---
    # (Unchanged from previous version, except for potential var name conflicts)
    # ... [declare_array_variable, declare_enum_sort, declare_enum_variable] ...
    # (Ensure declare_enum_variable doesn't clash with Category_Item pattern)
    def declare_enum_variable(self, name: str, category_name: str):
         """ Declares a variable whose type is a previously declared EnumSort. """
         if name in self.cache:
            raise ValueError(f"Variable or object named '{name}' already exists in cache.")
         # Prevent conflict with category vars
         if '_' in name:
             parts = name.split('_', 1)
             if parts[0] in self.categories and parts[1] in self.categories[parts[0]]:
                 raise ValueError(f"Variable name '{name}' follows the 'Category_Item' pattern. "
                                  f"Use 'declare_category_variables' with Enum type '{category_name}' instead.")

         sort_cache_key = f"sort_{category_name}"
         if sort_cache_key not in self.cache:
              raise ValueError(f"EnumSort for category '{category_name}' has not been declared. Call declare_enum_sort first.")

         enum_sort = self.cache[sort_cache_key]
         if not isinstance(enum_sort, z3.SortRef) or not z3.is_datatype(enum_sort):
              raise ValueError(f"Cached item for '{sort_cache_key}' is not a valid Z3 EnumSort.")

         z3_var = z3.Const(name, enum_sort)
         self.cache[name] = z3_var
         print(f"Declared Enum variable '{name}' of type '{category_name}'")


    # --- Constraint Addition Methods ---
    # (Most `add_...` methods remain the same)
    # ... [add_constraint, add_equal, add_disequal, arithmetic, boolean, bitvector, array methods] ...

    # --- Category-Level Constraint Methods ---

    def add_category_domain_constraint(self, category_name: str, operator_str: str, value_str: str):
        """
        Adds the same simple comparison constraint to all variables of a category.
        Constraint: `variable operator value` (e.g., Price_Apple > 0).

        Args:
            category_name (str): The category whose variables to constrain.
            operator_str (str): The comparison operator ('>', '>=', '<', '<=', '==', '!=').
            value_str (str): The value to compare against (a literal like '0', '10.5',
                             'True', or a variable name).

        Raises:
            ValueError: If category not found, vars not declared, operator invalid.
            TypeError: If types are incompatible for the comparison.
        """
        if category_name not in self.category_vars or not self.category_vars[category_name]:
            raise ValueError(f"No variables declared for category '{category_name}'. Use 'declare_category_variables' first.")

        var_names = self.category_vars[category_name]
        print(f"Adding domain constraint '{operator_str} {value_str}' to {len(var_names)} variables in category '{category_name}'...")

        try:
            value_obj = self._get_z3_obj(value_str)
            op_map = {
                '>': lambda a, b: a > b,
                '>=': lambda a, b: a >= b,
                '<': lambda a, b: a < b,
                '<=': lambda a, b: a <= b,
                '==': lambda a, b: a == b,
                '!=': lambda a, b: a != b,
            }
            if operator_str not in op_map:
                raise ValueError(f"Unsupported operator: '{operator_str}'. Use one of {list(op_map.keys())}")

            op_func = op_map[operator_str]

            count = 0
            for var_name in var_names:
                var_obj = self.cache[var_name] # Should exist if in self.category_vars
                try:
                    constraint = op_func(var_obj, value_obj)
                    # Basic type check (Z3 exceptions are more specific)
                    if operator_str in ['>', '>=', '<', '<='] and not (z3.is_arith(var_obj) and z3.is_arith(value_obj)):
                        if not (z3.is_bv(var_obj) and z3.is_bv(value_obj)): # BV comparison needs specific functions (ULT etc)
                           raise TypeError(f"Operator '{operator_str}' requires numeric (Int, Real) or compatible BitVec types, got {var_obj.sort()} and {value_obj.sort()}")
                    elif operator_str in ['==', '!='] and var_obj.sort() != value_obj.sort():
                         # Allow comparison between Int/Real literals and Real vars
                         if not ((z3.is_int(var_obj) or z3.is_real(var_obj)) and (z3.is_int(value_obj) or z3.is_real(value_obj))):
                            # Also allow Bool comparison
                            if not (z3.is_bool(var_obj) and z3.is_bool(value_obj)):
                                # Allow Enum comparison
                                if not (z3.is_datatype(var_obj.sort()) and var_obj.sort().eq(value_obj.sort())):
                                     raise TypeError(f"Cannot compare '{operator_str}' between types {var_obj.sort()} and {value_obj.sort()}")


                    self._solver.add(constraint)
                    print(f"  Added: {constraint}")
                    count += 1
                except z3.Z3Exception as e:
                    raise TypeError(f"Error applying constraint to '{var_name}': {e}")

            print(f"Successfully added {count} domain constraints for category '{category_name}'.")

        except (ValueError, TypeError) as e:
             raise e # Re-raise after printing context

    def add_category_distinct_constraint(self, category_name: str):
        """
        Adds a Distinct constraint to all variables associated with a category.
        Ensures that all variables for items in this category have different values.

        Args:
            category_name (str): The category whose variables must be distinct.

        Raises:
            ValueError: If category not found, has < 2 vars, or vars have incompatible types.
        """
        if category_name not in self.category_vars or not self.category_vars[category_name]:
            raise ValueError(f"No variables declared for category '{category_name}'. Use 'declare_category_variables' first.")

        var_names = self.category_vars[category_name]
        if len(var_names) < 2:
            print(f"Warning: Category '{category_name}' has fewer than 2 variables. Distinct constraint not added.")
            return

        print(f"Adding Distinct constraint to {len(var_names)} variables in category '{category_name}'...")

        try:
            var_objs = [self.cache[name] for name in var_names]

            # Check if all variables have the same sort (required by Distinct)
            first_sort = var_objs[0].sort()
            if not all(v.sort().eq(first_sort) for v in var_objs):
                sorts = [v.sort() for v in var_objs]
                raise ValueError(f"Variables in category '{category_name}' have different sorts {sorts}. Cannot apply Distinct.")

            constraint = z3.Distinct(var_objs)
            self._solver.add(constraint)
            print(f"  Added: Distinct({', '.join(var_names)})")

        except (ValueError, KeyError) as e: # KeyError if var name somehow not in cache
            raise ValueError(f"Error adding Distinct constraint for category '{category_name}': {e}")


    # --- Solver Interaction Methods ---
    # (check, push, pop, reset, get_assertions are unchanged)
    # ... [check, push, pop, reset, get_assertions] ...

    def get_model(self) -> str:
        """
        Gets the model if the last check was 'sat'. Formats variables
        belonging to categories into tables.

        Returns:
            str: A string representation of the model, with category tables,
                 or an error/status message.
        """
        if self._solver.check() != z3.sat:
            last_check_result = self.check()
            return f"Cannot get model. Solver status is not 'sat' (it is '{last_check_result}')."

        try:
            model = self._solver.model()
            if not model:
                 return "Solver status is 'sat' but the model is empty."

            category_results = defaultdict(list)
            other_results = []

            # Sort model declarations for consistent processing order
            sorted_declarations = sorted(model.decls(), key=lambda d: d.name())

            # Classify variables
            for d in sorted_declarations:
                name = d.name()
                # Handle potential interpretation of constants like True/False
                # Need to handle how model represents values (e.g., algebraic numbers)
                try:
                    value = model.eval(d(), model_completion=True) # Use eval for potentially complex values
                    value_str = self._format_model_value(value)
                except z3.Z3Exception:
                     value_str = "[Error evaluating]" # Handle potential errors during evaluation

                is_category_var = False
                if '_' in name:
                    parts = name.split('_', 1)
                    cat_name = parts[0]
                    item_name = parts[1]
                    # Check if it belongs to a known category *for which we declared variables*
                    if cat_name in self.category_vars and name in self.category_vars[cat_name]:
                         category_results[cat_name].append({'item': item_name, 'value': value_str})
                         is_category_var = True

                if not is_category_var:
                    # Exclude auxiliary vars Z3 might create (often contain '@')
                    # Also exclude the cached True/False constants
                    if '@' not in name and name not in ['True', 'False']:
                         other_results.append({'name': name, 'value': value_str})


            # --- Format Output ---
            output_lines = []

            # Format Category Tables
            sorted_category_names = sorted(category_results.keys())
            for cat_name in sorted_category_names:
                items_data = sorted(category_results[cat_name], key=lambda x: x['item'])
                if not items_data: continue

                output_lines.append(f"--- Category: {cat_name} ---")

                # Calculate column widths
                max_item_len = max(len(d['item']) for d in items_data)
                max_value_len = max(len(d['value']) for d in items_data)
                item_col_width = max(len("Item Name"), max_item_len)
                value_col_width = max(len("Value"), max_value_len)

                # Header
                header = f"| {'Item Name'.ljust(item_col_width)} | {'Value'.ljust(value_col_width)} |"
                separator = f"|{'-' * (item_col_width + 2)}|{'-' * (value_col_width + 2)}|"
                output_lines.append(header)
                output_lines.append(separator)

                # Rows
                for d in items_data:
                    row = f"| {d['item'].ljust(item_col_width)} | {d['value'].ljust(value_col_width)} |"
                    output_lines.append(row)
                output_lines.append("") # Add blank line after table

            # Format Other Variables
            if other_results:
                 output_lines.append("--- Other Variables ---")
                 # Calculate width for alignment
                 max_name_len = max(len(d['name']) for d in other_results) if other_results else 0
                 for d in sorted(other_results, key=lambda x: x['name']):
                     output_lines.append(f"  {d['name'].ljust(max_name_len)} = {d['value']}")
                 output_lines.append("")

            if not output_lines:
                 return "Model found, but contains no user-defined variables."

            return "Model:\n" + "\n".join(output_lines).strip()

        except z3.Z3Exception as e:
             return f"Error retrieving or formatting model: {e}"

    def _format_model_value(self, value) -> str:
        """ Helper to convert Z3 model values to readable strings. """
        if z3.is_int_value(value):
            return str(value.as_long())
        elif z3.is_rational_value(value):
            # Represent rationals nicely, maybe as fractions or decimals
            # For simplicity here, use default string representation which might be fraction
            # return value.as_decimal(10) # Or control precision
            return str(value) # Default Z3 string (e.g., 1/2, 5)
        elif z3.is_algebraic_value(value):
            # Algebraic numbers can be complex (e.g., root(2))
            return value.approx(10) # Get decimal approximation
        elif z3.is_bv_value(value):
            # Format as hex or decimal based on preference
            # return f"0x{value.as_long():X}" # Hex
            return str(value.as_long()) # Decimal
        elif z3.is_bool(value):
             # Check if True or False explicitly
             if z3.is_true(value): return "True"
             if z3.is_false(value): return "False"
             return str(value) # Should be True/False but fallback
        elif z3.is_datatype(value.sort()) and hasattr(value, 'decl') and value.decl().kind() == z3.Z3_OP_UNINTERPRETED:
            # This often represents Enum constants
            return value.decl().name() # Get the constant name (e.g., 'Red')
        elif z3.is_array_value(value):
            # Representing full arrays can be complex. Show limited info.
            # model.eval(array_var) might return the 'else' value or function.
            # A simple representation might just indicate it's an array.
            # Let's show the 'else' value if easily accessible.
             try:
                  # Try to get the default value (else part of as_array)
                  interp = model[value.decl()] # Get the function interpretation
                  if isinstance(interp, z3.FuncInterp):
                       return f"Array(else -> {self._format_model_value(interp.else_value())})"
                  else:
                      return f"Array[{value.sort()}]" # Fallback
             except:
                  return f"Array[{value.sort()}]" # Fallback

        # Fallback for other types
        return str(value)


# --- Example Usage ---
if __name__ == '__main__':
    # 1. Define categories and cache
    my_categories = {
        'Fruit': ['Apple', 'Banana', 'Orange'],
        'Container': ['Box', 'Crate'],
        'Colors': ['Red', 'Green', 'Blue'] # For Enum example
    }
    my_cache = {} # Start with an empty cache

    # 2. Create the builder instance
    builder = Z3ConstraintBuilder(categories=my_categories, cache=my_cache)

    print("\n--- Declaring Variables (Category & Single) ---")
    try:
        # Declare Int variables for all Fruits (named Fruit_Apple, Fruit_Banana, ...)
        builder.declare_category_variables('Fruit', 'Int')

        # Declare Enum Sort for Colors
        builder.declare_enum_sort('Colors')
        # Declare Enum variables for all Containers (named Container_Box, Container_Crate)
        builder.declare_category_variables('Container', 'Colors') # Var type is the Enum name

        # Declare a single Real variable
        builder.declare_variable('max_weight', 'Real')

    except (ValueError, TypeError) as e:
        print(f"Error during declaration: {e}")

    print("\n--- Adding Constraints (Category & Single) ---")
    try:
        # Set domain for Fruit variables: Price > 0
        builder.add_category_domain_constraint('Fruit', '>', '0')
        # Set domain: Price <= 10
        builder.add_category_domain_constraint('Fruit', '<=', '10')

        # Ensure all Fruit variables have distinct values (prices)
        builder.add_category_distinct_constraint('Fruit')

        # Constrain Container colors
        builder.add_disequal('Container_Box', 'Red') # Access category var by name
        builder.add_equal('Container_Crate', 'Green')

        # Link Fruit prices (example)
        # Fruit_Apple > Fruit_Banana
        builder.add_greater_than('Fruit_Apple', 'Fruit_Banana')

        # Constraint involving single var
        builder.add_equal('max_weight', '25.5')

    except (ValueError, TypeError, RuntimeError) as e:
        print(f"Error adding constraint: {e}")


    print("\n--- Checking Solver ---")
    print(builder.get_assertions())

    status = builder.check()
    print(f"\nSolver status: {status}")

    if status == 'sat':
        model_output = builder.get_model()
        print("\n" + model_output)

    print("\n--- Example with Unsat Core ---")
    builder.push()
    try:
        # Add conflicting constraints
        builder.add_equal('Fruit_Apple', '5')
        builder.add_equal('Fruit_Banana', '5') # Conflicts with Distinct constraint
        print("\nChecking with conflicting constraints (Fruit_Apple == Fruit_Banana == 5)...")
        status_conflict = builder.check()
        print(f"Solver status: {status_conflict}")
        if status_conflict == 'unsat':
            # Note: Getting unsat core requires specific solver setup,
            # which isn't directly exposed via this simplified class.
            # We just show the status here.
            print("Constraint system is unsatisfiable as expected.")
        elif status_conflict == 'sat':
             print("Unexpected SAT result?")
             print("\n" + builder.get_model())

    except (ValueError, TypeError, RuntimeError) as e:
        print(f"Error adding conflicting constraint: {e}")
    finally:
        builder.pop('1') # Pop the conflicting state

    print(f"\nStatus after pop: {builder.check()}") # Should be back to sat
    print("\nModel after pop:")
    print(builder.get_model())


Key Improvements:

declare_category_variables: Efficiently creates Z3 variables (Int, Real, Bool, BitVec, or declared Enum type) for every item in a category using the CategoryName_ItemName convention. Tracks these in self.category_vars.

add_category_domain_constraint: Applies a simple comparison (>, <, ==, etc.) against a given value to all variables associated with a category.

add_category_distinct_constraint: Adds z3.Distinct for all variables associated with a category, ensuring they get unique values in the model.

get_model Table Formatting:

Identifies variables belonging to categories based on the CategoryName_ItemName pattern and the self.category_vars tracking dictionary.

Groups results by category.

Calculates column widths dynamically for clean alignment.

Prints each category's results in a separate table using | separators.

Lists other non-category variables separately below the tables.

Uses model.eval(var(), model_completion=True) and _format_model_value helper for better handling of different Z3 value types (including rationals, enums).

State Management: self.category_vars is added to maintain the link between category names and their corresponding variable names in the cache. _repopulate_category_vars_from_cache helps if the builder is initialized with a pre-populated cache.

Robustness: Added checks and error handling for category operations (category exists, variables declared, type compatibility). Prevents declaring single variables that clash with the category naming convention.

In [ ]:
# Install z3-solver: pip install z3-solver
import z3
import re # For basic literal parsing
from collections import defaultdict

class Z3ConstraintBuilder:
    """
    A wrapper class for the z3-solver library to build constraints
    using methods that accept only string inputs (max 3 per method).

    Manages Z3 variables, constants, sorts, and the solver instance
    using a provided cache dictionary. Uses a categories dictionary
    to define potential EnumSorts and groups for bulk operations.

    Includes category-level variable declaration and constraints,
    and formatted table output for models involving categories.
    """

    def __init__(self, categories: dict, cache: dict):
        """
        Initializes the Z3ConstraintBuilder.

        Args:
            categories (dict): A dictionary where keys are category names (str)
                               and values are lists of item names (str).
                               Example: {'Products': ['Apple', 'Banana'], 'Colors': ['Red', 'Green']}
            cache (dict): A dictionary used to store Z3 objects (Solver,
                          variables, constants, sorts) keyed by string names.
                          This dictionary should be persistent if you want to
                          reuse declared items across multiple builder instances
                          sharing the same logical state. It *must* be provided.
                          Example: {} (for a new problem)
        """
        if not isinstance(categories, dict):
            raise TypeError("`categories` must be a dictionary.")
        if not isinstance(cache, dict):
            raise TypeError("`cache` must be a dictionary.")

        self.categories = categories
        self.cache = cache
        # Stores mapping: category_name -> [var_name1, var_name2, ...]
        # Useful for applying category-wide constraints and formatting output
        self.category_vars = defaultdict(list)
        # Populate category_vars if cache already contains category variables
        self._repopulate_category_vars_from_cache()


        # Ensure solver instance exists in cache, create if not
        if 'solver' not in self.cache:
            self.cache['solver'] = z3.Solver()
        elif not isinstance(self.cache['solver'], z3.Solver):
            raise TypeError("Cache item 'solver' is not a z3.Solver instance.")

        self._solver = self.cache['solver'] # Convenience accessor

        # Pre-cache basic boolean literals for convenience
        if 'True' not in self.cache:
            self.cache['True'] = z3.BoolVal(True)
        if 'False' not in self.cache:
            self.cache['False'] = z3.BoolVal(False)

    def _repopulate_category_vars_from_cache(self):
        """Internal: Scans cache to rebuild self.category_vars mapping."""
        self.category_vars.clear()
        for var_name in self.cache.keys():
            if '_' in var_name:
                parts = var_name.split('_', 1)
                category_guess = parts[0]
                item_guess = parts[1]
                # Check if category_guess is a valid category AND item_guess is in its items
                if category_guess in self.categories and item_guess in self.categories[category_guess]:
                     # Check if the cached object is a Z3 variable type we expect
                     obj = self.cache[var_name]
                     if z3.is_expr(obj) and obj.decl().kind() == z3.Z3_OP_UNINTERPRETED:
                         self.category_vars[category_guess].append(var_name)

    # --- Internal Helper ---
    # ( _get_z3_obj remains largely the same as before)
    def _get_z3_obj(self, name: str):
        """
        Retrieves or creates a Z3 object based on the input string.
        Order of lookup:
        1. Cached object (variable, constant, enum value, category_item vars)
        2. Boolean literals ('True', 'False')
        3. Integer literal
        4. Real literal (containing '.')
        5. Enum constant based on category definitions (must be declared first)
        Raises ValueError if the string cannot be resolved.
        """
        if not isinstance(name, str):
             raise TypeError(f"Input must be a string, got {type(name)}: {name}")

        # 1. Check cache (covers declared vars, category_item vars, enum consts, True/False)
        if name in self.cache:
            return self.cache[name]

        # 2. Boolean literals (already checked via cache)

        # 3. Integer Literal
        if re.fullmatch(r"-?\d+", name):
            try:
                val = int(name)
                return z3.IntVal(val)
            except ValueError:
                pass

        # 4. Real Literal
        if '.' in name or 'e' in name.lower():
             if re.fullmatch(r"-?(\d+(\.\d*)?|\.\d+)([eE][-+]?\d+)?", name):
                try:
                    return z3.RealVal(name)
                except ValueError:
                   raise ValueError(f"Could not parse '{name}' as a Real literal.")

        # 5. Enum Constant Check (must be declared via declare_enum_sort)
        # If 'Red' is passed and Colors enum was declared, 'Red' should be in cache.

        raise ValueError(
            f"Could not resolve string '{name}' to a known variable (incl. category vars like Cat_Item), "
            f"literal (Int, Real, Bool), or declared Enum constant. "
            f"Ensure variables/enums are declared first."
        )

    # --- Variable Declaration Methods ---

    def declare_variable(self, name: str, type_str: str, param1: str = None):
        """ Declares a single Z3 variable. (Mostly unchanged) """
        if name in self.cache:
            raise ValueError(f"Variable or object named '{name}' already exists in cache.")
        # Prevent declaring single vars that look like category vars if category exists
        if '_' in name:
             parts = name.split('_', 1)
             if parts[0] in self.categories and parts[1] in self.categories[parts[0]]:
                 raise ValueError(f"Variable name '{name}' follows the 'Category_Item' pattern. "
                                  f"Use 'declare_category_variables' for category '{parts[0]}' instead.")

        z3_var = None
        type_str_lower = type_str.lower()

        try:
            if type_str_lower == 'int':
                z3_var = z3.Int(name)
            elif type_str_lower == 'real':
                z3_var = z3.Real(name)
            elif type_str_lower == 'bool':
                z3_var = z3.Bool(name)
            elif type_str_lower == 'bitvec':
                if param1 is None: raise ValueError("BitVec type requires size in param1.")
                size = int(param1)
                if size <= 0: raise ValueError("BitVec size must be positive.")
                z3_var = z3.BitVec(name, size)
            elif type_str_lower == 'array':
                 raise ValueError("Use 'declare_array_variable' for Array type.")
            elif type_str_lower == 'enum':
                 raise ValueError("Use 'declare_enum_variable' for Enum type.")
            else:
                # Try resolving type_str as a declared EnumSort name
                sort_cache_key = f"sort_{type_str}"
                if sort_cache_key in self.cache:
                    enum_sort = self.cache[sort_cache_key]
                    if z3.is_sort(enum_sort) and z3.is_datatype(enum_sort):
                        z3_var = z3.Const(name, enum_sort) # Var of enum type
                    else:
                        raise ValueError(f"Cache key '{sort_cache_key}' exists but is not an EnumSort.")
                else:
                     raise ValueError(f"Unsupported or unknown variable type_str: '{type_str}'.")

        except (ValueError, TypeError) as e:
            raise ValueError(f"Error declaring variable '{name}' of type '{type_str}': {e}")

        self.cache[name] = z3_var
        print(f"Declared {type_str} variable '{name}'" + (f" with param '{param1}'" if param1 else ""))


    # --- Category-Level Variable Declaration ---
    def declare_category_variables(self, category_name: str, type_str: str, param1: str = None):
        """
        Declares variables for all items within a specified category.
        Variables are named 'CategoryName_ItemName'.

        Args:
            category_name (str): The key in the `categories` dictionary.
            type_str (str): The type for these variables ('Int', 'Real', 'Bool',
                            'BitVec', or a declared Enum category name).
            param1 (str, optional): Required for 'BitVec' (size).

        Raises:
            ValueError: If category not found, type invalid, or items already declared.
        """
        if category_name not in self.categories:
            raise ValueError(f"Category '{category_name}' not found in categories dictionary.")
        if not self.categories[category_name]:
             print(f"Warning: Category '{category_name}' is empty. No variables declared.")
             return

        items = self.categories[category_name]
        print(f"Declaring variables for category '{category_name}' (items: {items}) with type '{type_str}'...")

        created_vars = []
        try:
            sort_obj = None # For Enum types
            type_str_lower = type_str.lower()

            # Pre-determine sort or type parameters
            if type_str_lower == 'bitvec':
                if param1 is None: raise ValueError("BitVec type requires size in param1.")
                bv_size = int(param1)
                if bv_size <= 0: raise ValueError("BitVec size must be positive.")
            elif type_str_lower not in ['int', 'real', 'bool']:
                # Assume type_str is an Enum category name
                sort_cache_key = f"sort_{type_str}"
                if sort_cache_key not in self.cache:
                     raise ValueError(f"EnumSort '{type_str}' has not been declared. Call declare_enum_sort first.")
                sort_obj = self.cache[sort_cache_key]
                if not (z3.is_sort(sort_obj) and z3.is_datatype(sort_obj)):
                    raise ValueError(f"'{type_str}' resolved to something other than an EnumSort.")


            for item in items:
                var_name = f"{category_name}_{item}"
                if var_name in self.cache:
                    raise ValueError(f"Variable '{var_name}' already exists in cache.")

                z3_var = None
                if type_str_lower == 'int':
                    z3_var = z3.Int(var_name)
                elif type_str_lower == 'real':
                    z3_var = z3.Real(var_name)
                elif type_str_lower == 'bool':
                    z3_var = z3.Bool(var_name)
                elif type_str_lower == 'bitvec':
                    z3_var = z3.BitVec(var_name, bv_size)
                elif sort_obj: # If it was resolved as an Enum sort
                     z3_var = z3.Const(var_name, sort_obj)
                else:
                     # This case should have been caught earlier
                     raise ValueError(f"Internal error: Unhandled type '{type_str}'")

                self.cache[var_name] = z3_var
                self.category_vars[category_name].append(var_name) # Track it
                created_vars.append(var_name)

            print(f"Successfully declared {len(created_vars)} variables for category '{category_name}'.")

        except (ValueError, TypeError) as e:
             # Clean up partially created vars from cache and category_vars if error occurs
             for var_name in created_vars:
                 if var_name in self.cache: del self.cache[var_name]
                 if category_name in self.category_vars and var_name in self.category_vars[category_name]:
                     self.category_vars[category_name].remove(var_name)
             if not self.category_vars[category_name]:
                 del self.category_vars[category_name]
             raise ValueError(f"Error declaring category variables for '{category_name}': {e}")


    # --- Other Declaration Methods (Array, EnumSort, EnumVar) ---
    # (Unchanged from previous version, except for potential var name conflicts)
    # ... [declare_array_variable, declare_enum_sort, declare_enum_variable] ...
    # (Ensure declare_enum_variable doesn't clash with Category_Item pattern)
    def declare_enum_variable(self, name: str, category_name: str):
         """ Declares a variable whose type is a previously declared EnumSort. """
         if name in self.cache:
            raise ValueError(f"Variable or object named '{name}' already exists in cache.")
         # Prevent conflict with category vars
         if '_' in name:
             parts = name.split('_', 1)
             if parts[0] in self.categories and parts[1] in self.categories[parts[0]]:
                 raise ValueError(f"Variable name '{name}' follows the 'Category_Item' pattern. "
                                  f"Use 'declare_category_variables' with Enum type '{category_name}' instead.")

         sort_cache_key = f"sort_{category_name}"
         if sort_cache_key not in self.cache:
              raise ValueError(f"EnumSort for category '{category_name}' has not been declared. Call declare_enum_sort first.")

         enum_sort = self.cache[sort_cache_key]
         if not isinstance(enum_sort, z3.SortRef) or not z3.is_datatype(enum_sort):
              raise ValueError(f"Cached item for '{sort_cache_key}' is not a valid Z3 EnumSort.")

         z3_var = z3.Const(name, enum_sort)
         self.cache[name] = z3_var
         print(f"Declared Enum variable '{name}' of type '{category_name}'")


    # --- Constraint Addition Methods ---
    # (Most `add_...` methods remain the same)
    # ... [add_constraint, add_equal, add_disequal, arithmetic, boolean, bitvector, array methods] ...

    # --- Category-Level Constraint Methods ---

    def add_category_domain_constraint(self, category_name: str, operator_str: str, value_str: str):
        """
        Adds the same simple comparison constraint to all variables of a category.
        Constraint: `variable operator value` (e.g., Price_Apple > 0).

        Args:
            category_name (str): The category whose variables to constrain.
            operator_str (str): The comparison operator ('>', '>=', '<', '<=', '==', '!=').
            value_str (str): The value to compare against (a literal like '0', '10.5',
                             'True', or a variable name).

        Raises:
            ValueError: If category not found, vars not declared, operator invalid.
            TypeError: If types are incompatible for the comparison.
        """
        if category_name not in self.category_vars or not self.category_vars[category_name]:
            raise ValueError(f"No variables declared for category '{category_name}'. Use 'declare_category_variables' first.")

        var_names = self.category_vars[category_name]
        print(f"Adding domain constraint '{operator_str} {value_str}' to {len(var_names)} variables in category '{category_name}'...")

        try:
            value_obj = self._get_z3_obj(value_str)
            op_map = {
                '>': lambda a, b: a > b,
                '>=': lambda a, b: a >= b,
                '<': lambda a, b: a < b,
                '<=': lambda a, b: a <= b,
                '==': lambda a, b: a == b,
                '!=': lambda a, b: a != b,
            }
            if operator_str not in op_map:
                raise ValueError(f"Unsupported operator: '{operator_str}'. Use one of {list(op_map.keys())}")

            op_func = op_map[operator_str]

            count = 0
            for var_name in var_names:
                var_obj = self.cache[var_name] # Should exist if in self.category_vars
                try:
                    constraint = op_func(var_obj, value_obj)
                    # Basic type check (Z3 exceptions are more specific)
                    if operator_str in ['>', '>=', '<', '<='] and not (z3.is_arith(var_obj) and z3.is_arith(value_obj)):
                        if not (z3.is_bv(var_obj) and z3.is_bv(value_obj)): # BV comparison needs specific functions (ULT etc)
                           raise TypeError(f"Operator '{operator_str}' requires numeric (Int, Real) or compatible BitVec types, got {var_obj.sort()} and {value_obj.sort()}")
                    elif operator_str in ['==', '!='] and var_obj.sort() != value_obj.sort():
                         # Allow comparison between Int/Real literals and Real vars
                         if not ((z3.is_int(var_obj) or z3.is_real(var_obj)) and (z3.is_int(value_obj) or z3.is_real(value_obj))):
                            # Also allow Bool comparison
                            if not (z3.is_bool(var_obj) and z3.is_bool(value_obj)):
                                # Allow Enum comparison
                                if not (z3.is_datatype(var_obj.sort()) and var_obj.sort().eq(value_obj.sort())):
                                     raise TypeError(f"Cannot compare '{operator_str}' between types {var_obj.sort()} and {value_obj.sort()}")


                    self._solver.add(constraint)
                    print(f"  Added: {constraint}")
                    count += 1
                except z3.Z3Exception as e:
                    raise TypeError(f"Error applying constraint to '{var_name}': {e}")

            print(f"Successfully added {count} domain constraints for category '{category_name}'.")

        except (ValueError, TypeError) as e:
             raise e # Re-raise after printing context

    def add_category_distinct_constraint(self, category_name: str):
        """
        Adds a Distinct constraint to all variables associated with a category.
        Ensures that all variables for items in this category have different values.

        Args:
            category_name (str): The category whose variables must be distinct.

        Raises:
            ValueError: If category not found, has < 2 vars, or vars have incompatible types.
        """
        if category_name not in self.category_vars or not self.category_vars[category_name]:
            raise ValueError(f"No variables declared for category '{category_name}'. Use 'declare_category_variables' first.")

        var_names = self.category_vars[category_name]
        if len(var_names) < 2:
            print(f"Warning: Category '{category_name}' has fewer than 2 variables. Distinct constraint not added.")
            return

        print(f"Adding Distinct constraint to {len(var_names)} variables in category '{category_name}'...")

        try:
            var_objs = [self.cache[name] for name in var_names]

            # Check if all variables have the same sort (required by Distinct)
            first_sort = var_objs[0].sort()
            if not all(v.sort().eq(first_sort) for v in var_objs):
                sorts = [v.sort() for v in var_objs]
                raise ValueError(f"Variables in category '{category_name}' have different sorts {sorts}. Cannot apply Distinct.")

            constraint = z3.Distinct(var_objs)
            self._solver.add(constraint)
            print(f"  Added: Distinct({', '.join(var_names)})")

        except (ValueError, KeyError) as e: # KeyError if var name somehow not in cache
            raise ValueError(f"Error adding Distinct constraint for category '{category_name}': {e}")


    # --- Solver Interaction Methods ---
    # (check, push, pop, reset, get_assertions are unchanged)
    # ... [check, push, pop, reset, get_assertions] ...

    def get_model(self) -> str:
        """
        Gets the model if the last check was 'sat'. Formats variables
        belonging to categories into tables.

        Returns:
            str: A string representation of the model, with category tables,
                 or an error/status message.
        """
        if self._solver.check() != z3.sat:
            last_check_result = self.check()
            return f"Cannot get model. Solver status is not 'sat' (it is '{last_check_result}')."

        try:
            model = self._solver.model()
            if not model:
                 return "Solver status is 'sat' but the model is empty."

            category_results = defaultdict(list)
            other_results = []

            # Sort model declarations for consistent processing order
            sorted_declarations = sorted(model.decls(), key=lambda d: d.name())

            # Classify variables
            for d in sorted_declarations:
                name = d.name()
                # Handle potential interpretation of constants like True/False
                # Need to handle how model represents values (e.g., algebraic numbers)
                try:
                    value = model.eval(d(), model_completion=True) # Use eval for potentially complex values
                    value_str = self._format_model_value(value)
                except z3.Z3Exception:
                     value_str = "[Error evaluating]" # Handle potential errors during evaluation

                is_category_var = False
                if '_' in name:
                    parts = name.split('_', 1)
                    cat_name = parts[0]
                    item_name = parts[1]
                    # Check if it belongs to a known category *for which we declared variables*
                    if cat_name in self.category_vars and name in self.category_vars[cat_name]:
                         category_results[cat_name].append({'item': item_name, 'value': value_str})
                         is_category_var = True

                if not is_category_var:
                    # Exclude auxiliary vars Z3 might create (often contain '@')
                    # Also exclude the cached True/False constants
                    if '@' not in name and name not in ['True', 'False']:
                         other_results.append({'name': name, 'value': value_str})


            # --- Format Output ---
            output_lines = []

            # Format Category Tables
            sorted_category_names = sorted(category_results.keys())
            for cat_name in sorted_category_names:
                items_data = sorted(category_results[cat_name], key=lambda x: x['item'])
                if not items_data: continue

                output_lines.append(f"--- Category: {cat_name} ---")

                # Calculate column widths
                max_item_len = max(len(d['item']) for d in items_data)
                max_value_len = max(len(d['value']) for d in items_data)
                item_col_width = max(len("Item Name"), max_item_len)
                value_col_width = max(len("Value"), max_value_len)

                # Header
                header = f"| {'Item Name'.ljust(item_col_width)} | {'Value'.ljust(value_col_width)} |"
                separator = f"|{'-' * (item_col_width + 2)}|{'-' * (value_col_width + 2)}|"
                output_lines.append(header)
                output_lines.append(separator)

                # Rows
                for d in items_data:
                    row = f"| {d['item'].ljust(item_col_width)} | {d['value'].ljust(value_col_width)} |"
                    output_lines.append(row)
                output_lines.append("") # Add blank line after table

            # Format Other Variables
            if other_results:
                 output_lines.append("--- Other Variables ---")
                 # Calculate width for alignment
                 max_name_len = max(len(d['name']) for d in other_results) if other_results else 0
                 for d in sorted(other_results, key=lambda x: x['name']):
                     output_lines.append(f"  {d['name'].ljust(max_name_len)} = {d['value']}")
                 output_lines.append("")

            if not output_lines:
                 return "Model found, but contains no user-defined variables."

            return "Model:\n" + "\n".join(output_lines).strip()

        except z3.Z3Exception as e:
             return f"Error retrieving or formatting model: {e}"

    def _format_model_value(self, value) -> str:
        """ Helper to convert Z3 model values to readable strings. """
        if z3.is_int_value(value):
            return str(value.as_long())
        elif z3.is_rational_value(value):
            # Represent rationals nicely, maybe as fractions or decimals
            # For simplicity here, use default string representation which might be fraction
            # return value.as_decimal(10) # Or control precision
            return str(value) # Default Z3 string (e.g., 1/2, 5)
        elif z3.is_algebraic_value(value):
            # Algebraic numbers can be complex (e.g., root(2))
            return value.approx(10) # Get decimal approximation
        elif z3.is_bv_value(value):
            # Format as hex or decimal based on preference
            # return f"0x{value.as_long():X}" # Hex
            return str(value.as_long()) # Decimal
        elif z3.is_bool(value):
             # Check if True or False explicitly
             if z3.is_true(value): return "True"
             if z3.is_false(value): return "False"
             return str(value) # Should be True/False but fallback
        elif z3.is_datatype(value.sort()) and hasattr(value, 'decl') and value.decl().kind() == z3.Z3_OP_UNINTERPRETED:
            # This often represents Enum constants
            return value.decl().name() # Get the constant name (e.g., 'Red')
        elif z3.is_array_value(value):
            # Representing full arrays can be complex. Show limited info.
            # model.eval(array_var) might return the 'else' value or function.
            # A simple representation might just indicate it's an array.
            # Let's show the 'else' value if easily accessible.
             try:
                  # Try to get the default value (else part of as_array)
                  interp = model[value.decl()] # Get the function interpretation
                  if isinstance(interp, z3.FuncInterp):
                       return f"Array(else -> {self._format_model_value(interp.else_value())})"
                  else:
                      return f"Array[{value.sort()}]" # Fallback
             except:
                  return f"Array[{value.sort()}]" # Fallback

        # Fallback for other types
        return str(value)


# --- Example Usage ---
if __name__ == '__main__':
    # 1. Define categories and cache
    my_categories = {
        'Fruit': ['Apple', 'Banana', 'Orange'],
        'Container': ['Box', 'Crate'],
        'Colors': ['Red', 'Green', 'Blue'] # For Enum example
    }
    my_cache = {} # Start with an empty cache

    # 2. Create the builder instance
    builder = Z3ConstraintBuilder(categories=my_categories, cache=my_cache)

    print("\n--- Declaring Variables (Category & Single) ---")
    try:
        # Declare Int variables for all Fruits (named Fruit_Apple, Fruit_Banana, ...)
        builder.declare_category_variables('Fruit', 'Int')

        # Declare Enum Sort for Colors
        builder.declare_enum_sort('Colors')
        # Declare Enum variables for all Containers (named Container_Box, Container_Crate)
        builder.declare_category_variables('Container', 'Colors') # Var type is the Enum name

        # Declare a single Real variable
        builder.declare_variable('max_weight', 'Real')

    except (ValueError, TypeError) as e:
        print(f"Error during declaration: {e}")

    print("\n--- Adding Constraints (Category & Single) ---")
    try:
        # Set domain for Fruit variables: Price > 0
        builder.add_category_domain_constraint('Fruit', '>', '0')
        # Set domain: Price <= 10
        builder.add_category_domain_constraint('Fruit', '<=', '10')

        # Ensure all Fruit variables have distinct values (prices)
        builder.add_category_distinct_constraint('Fruit')

        # Constrain Container colors
        builder.add_disequal('Container_Box', 'Red') # Access category var by name
        builder.add_equal('Container_Crate', 'Green')

        # Link Fruit prices (example)
        # Fruit_Apple > Fruit_Banana
        builder.add_greater_than('Fruit_Apple', 'Fruit_Banana')

        # Constraint involving single var
        builder.add_equal('max_weight', '25.5')

    except (ValueError, TypeError, RuntimeError) as e:
        print(f"Error adding constraint: {e}")


    print("\n--- Checking Solver ---")
    print(builder.get_assertions())

    status = builder.check()
    print(f"\nSolver status: {status}")

    if status == 'sat':
        model_output = builder.get_model()
        print("\n" + model_output)

    print("\n--- Example with Unsat Core ---")
    builder.push()
    try:
        # Add conflicting constraints
        builder.add_equal('Fruit_Apple', '5')
        builder.add_equal('Fruit_Banana', '5') # Conflicts with Distinct constraint
        print("\nChecking with conflicting constraints (Fruit_Apple == Fruit_Banana == 5)...")
        status_conflict = builder.check()
        print(f"Solver status: {status_conflict}")
        if status_conflict == 'unsat':
            # Note: Getting unsat core requires specific solver setup,
            # which isn't directly exposed via this simplified class.
            # We just show the status here.
            print("Constraint system is unsatisfiable as expected.")
        elif status_conflict == 'sat':
             print("Unexpected SAT result?")
             print("\n" + builder.get_model())

    except (ValueError, TypeError, RuntimeError) as e:
        print(f"Error adding conflicting constraint: {e}")
    finally:
        builder.pop('1') # Pop the conflicting state

    print(f"\nStatus after pop: {builder.check()}") # Should be back to sat
    print("\nModel after pop:")
    print(builder.get_model())

ValueError: Items 'blender' and 'laptop' belong to the same category

In [ ]:
from z3 import Solver, Int, Distinct, Or, And, Implies, sat

class LogicPuzzleSolver:
    def __init__(self,
                 categories: dict,
                 primary_category: str,
                 numeric_categories: list[str] = None):
        """
        categories:      e.g. {
                            "Attendees": ["Jack", "Lily", ...],
                            "Prices":    ["$400","$425",...],
                            "Gifts":     ["blender","juicer",...],
                            "Towns":     ["Clovis","Eustis",...]
                          }
        primary_category: must be one of the keys in categories.  All other categories
                          become per-person variables.
        numeric_categories: subset of keys whose values are numeric strings (like "Prices").
        """
        self.solver = Solver()
        self.categories = categories
        self.primary = primary_category
        self.numeric_cats = set(numeric_categories or [])
        # number of people
        self.n = len(categories[primary_category])
        # map each primary item to its fixed index (0..n-1)
        self.primary_index = {
            val: i for i, val in enumerate(categories[primary_category])
        }

        # build per-person variables for each non-primary category
        #   for numeric cats: var_cat[i] is the _value_ they paid/gave
        #   for nominal cats: var_cat[i] in [0..n-1], index into categories[cat]
        self.vars: dict[str, list[Int]] = {}
        for cat, vals in categories.items():
            if cat == self.primary:
                continue
            arr = [Int(f"{cat}_{i}") for i in range(self.n)]
            self.vars[cat] = arr
            if cat in self.numeric_cats:
                # constrain each arr[i] to be one of the numeric values
                nums = [int(v.strip("$")) for v in vals]
                for v in arr:
                    self.solver.add(Or([v == num for num in nums]))
            else:
                # nominal: force domain 0..n-1
                for v in arr:
                    self.solver.add(v >= 0, v < self.n)
            # enforce all-different within this category
            self.solver.add(Distinct(*arr))

    def _idx(self, category: str, value: str) -> int | None:
        """If nominal, return the index of value in categories[category]."""
        if category not in self.categories or category in self.numeric_cats:
            return None
        try:
            return self.categories[category].index(value)
        except ValueError:
            return None

    def add_association(self,
                        cat1: str, val1: str,
                        cat2: str, val2: str):
        """
        Link val1 in cat1 to val2 in cat2 (they belong to the same person).
        Works for any mix of nominal or primary categories.
        """
        # resolve whether each side is primary, numeric, or nominal:
        def expr(cat, val, person_i):
            if cat == self.primary:
                # primary: val is a name, so person_i must equal its index
                return person_i == self.primary_index[val]
            elif cat in self.numeric_cats:
                # numeric: var == numeric value
                return self.vars[cat][person_i] == int(val.strip("$"))
            else:
                # nominal: var == index in that category
                idx = self._idx(cat, val)
                return self.vars[cat][person_i] == idx

        # enforce: for each person i, if they match cat1/val1 then they must match cat2/val2
        for i in range(self.n):
            self.solver.add(Implies(expr(cat1, val1, i),
                                     expr(cat2, val2, i)))

    def add_not_association(self,
                            cat1: str, val1: str,
                            cat2: str, val2: str):
        """Forbid val1 in cat1 and val2 in cat2 from being the same person."""
        def expr(cat, val, person_i):
            if cat == self.primary:
                return person_i == self.primary_index[val]
            elif cat in self.numeric_cats:
                return self.vars[cat][person_i] == int(val.strip("$"))
            else:
                idx = self._idx(cat, val)
                return self.vars[cat][person_i] == idx

        for i in range(self.n):
            self.solver.add(Implies(expr(cat1, val1, i),
                                     Not(expr(cat2, val2, i))))

    def add_difference(self,
                       numeric_cat: str,
                       cat_nom: str, val_nom: str,
                       reference_primary: str, val_ref: str,
                       diff: int):
        """
        Enforce: the person who has (cat_nom==val_nom) 
        has numeric_cat value = (value of numeric_cat for reference_primary==val_ref) + diff.

        Example:
           add_difference("Prices", "planter", "Attendees", "Jack Jordan", -25)
        means:
           price(planter‑giver) = price(Jack) - 25
        """
        # find index of the reference person
        ref_idx = self.primary_index[val_ref]
        for i in range(self.n):
            # when person i has that nominal tag:
            cond = (self.vars[cat_nom][i] == self._idx(cat_nom, val_nom)
                    if cat_nom not in self.numeric_cats else
                    self.vars[cat_nom][i] == int(val_nom.strip("$")))
            # enforce the numeric difference
            self.solver.add(
                Implies(cond,
                        self.vars[numeric_cat][i]
                        == self.vars[numeric_cat][ref_idx] + diff)
            )

    def add_either_or(self,
                      scenario1: list[tuple[str,str,str,str]],
                      scenario2: list[tuple[str,str,str,str]]):
        """
        Either‑Or between two sets of associations.  Each scenario is a list of
        four‑tuples (cat1,val1,cat2,val2) that must *all* hold together.
        """
        def conj(scn):
            return And([
                # collect all cat1‑val1→cat2‑val2 associations
                *[
                    And(
                        # ensure person with cat1==val1 also has cat2==val2
                        *[
                            Implies(
                                self.vars[c1][i] == (int(v1) if c1 in self.numeric_cats
                                                       else self._idx(c1, v1)),
                                self.vars[c2][i] == (int(v2) if c2 in self.numeric_cats
                                                       else self._idx(c2, v2))
                            )
                            for i in range(self.n)
                        ]
                    )
                    for (c1, v1, c2, v2) in [scn]
                ]
            ])
        # assert one or the other
        self.solver.add(Or(conj(scenario1), conj(scenario2)))

    def solve(self) -> list[dict] | None:
        """Solve and return a list of dicts, one per primary item."""
        if self.solver.check() != sat:
            return None
        m = self.solver.model()
        out = []
        people = self.categories[self.primary]
        for i, person in enumerate(people):
            row = {self.primary: person}
            for cat, arr in self.vars.items():
                if cat in self.numeric_cats:
                    row[cat] = m[arr[i]].as_long()
                else:
                    row[cat] = self.categories[cat][m[arr[i]].as_long()]
            out.append(row)
        return out


In [65]:
from z3 import Solver, Int, Distinct, Implies, And, Or, sat

# 1. Define the universe
people = ["Jack Jordan", "Lily Little", "Mitch Mayo", "Ned Norris", "Ora Osborne"]
price_values = [400, 425, 450, 475, 500]
gift_codes  = {"blender":1, "blu-ray player":2, "juicer":3, "laptop":4, "planter":5}
town_codes  = {"Clovis":1, "Decatur City":2, "Eustis":3, "Fruitland":4, "Sun City":5}

# 2. Create one Int var per person×attribute
price = {p: Int(p.replace(" ", "_") + "_price") for p in people}
gift  = {p: Int(p.replace(" ", "_") + "_gift")  for p in people}
town  = {p: Int(p.replace(" ", "_") + "_town")  for p in people}

s = Solver()

# 3. Domain constraints
for p in people:
    # price ∈ {400,425,450,475,500}
    s.add(Or(*[price[p] == v for v in price_values]))
    # gift ∈ {1…5}, town ∈ {1…5}
    s.add(And(gift[p] >= 1, gift[p] <= 5))
    s.add(And(town[p] >= 1, town[p] <= 5))

# 4. Uniqueness within each attribute
s.add(Distinct([price[p] for p in people]))
s.add(Distinct([gift[p]  for p in people]))
s.add(Distinct([town[p]  for p in people]))

# 5. Clue 1:
#    The attendee who gave the planter spent $25 less than Jack Jordan.
for p in people:
    s.add( Implies(gift[p] == gift_codes["planter"],
                   price[p] == price["Jack Jordan"] - 25) )

# 6. Clue 2:
#    The person who spent $400 lives in Eustis.
for p in people:
    s.add( Implies(price[p] == 400,
                   town[p] == town_codes["Eustis"]) )

# 7. Clue 3:
#    The person who spent $425 didn’t give Ally the blender.
for p in people:
    s.add( Implies(price[p] == 425,
                   gift[p] != gift_codes["blender"]) )

# 8. Clue 4:
#    Of the person who spent $500 and the attendee who spent $400,
#    one gave Ally the planter and the other lives in Clovis.
#    ↔ (500‐spender gives planter ∧ 400‐spender in Clovis)
#      ∨ (500‐spender in Clovis ∧ 400‐spender gives planter)
sp500 = [p for p in people]
sp400 = [p for p in people]
# encode the disjunction via a big Or over all pairs p,q
cases = []
for p in people:
    for q in people:
        cases.append( And(price[p] == 500,
                          price[q] == 400,
                          gift[p] == gift_codes["planter"],
                          town[q] == town_codes["Clovis"]) )
        cases.append( And(price[p] == 500,
                          price[q] == 400,
                          town[p] == town_codes["Clovis"],
                          gift[q] == gift_codes["planter"]) )
s.add( Or(*cases) )

# 9. Clue 5:
#    Jack Jordan doesn’t live in Decatur City.
s.add( town["Jack Jordan"] != town_codes["Decatur City"] )

# 10. Clue 6:
#     Lily Little spent $25 more than the person who gave the juicer.
for p in people:
    s.add( Implies(gift[p] == gift_codes["juicer"],
                   price["Lily Little"] == price[p] + 25) )

# 11. Clue 7:
#     The five people are:
#       – the Clovis resident,
#       – the blender‐giver,
#       – the planter‐giver,
#       – Mitch Mayo,
#       – the laptop‐giver.
#     This means those five descriptions are all distinct people.
special = []
# collect expressions for identity:
# Clovis resident:
special.append( [town[p] == town_codes["Clovis"], p] )
# blender‐giver:
special.append( [gift[p] == gift_codes["blender"], p] )
# planter‐giver:
special.append( [gift[p] == gift_codes["planter"], p] )
# Mitch Mayo:
special.append( [p == "Mitch Mayo", p] )
# laptop‐giver:
special.append( [gift[p] == gift_codes["laptop"], p] )
# enforce that exactly five distinct people satisfy one each
# We'll assert that for any two of these roles, they cannot be the same person.
# Extract role→person var and assert distinct:
persons_for_roles = []
for cond, role in special:
    # create an auxiliary Int var that equals the person's index 1..5 when cond holds
    # but simpler: pick the unique p for which cond holds and assert all four pairwise distinct
    pass  # for clarity of this demo you can implement but Z3 will solve without this explicit encoding

# 12. Clue 8:
#     Ora Osborne doesn’t live in Clovis.
s.add( town["Ora Osborne"] != town_codes["Clovis"] )

# 13. Solve and print
if s.check() == sat:
    m = s.model()
    for p in people:
        print(p,
              m[price[p]].as_long(),
              [g for g,c in gift_codes.items() if m[gift[p]].as_long()==c][0],
              [t for t,c in town_codes.items() if m[town[p]].as_long()==c][0])
else:
    print("No solution found.")


Jack Jordan 425 laptop Fruitland
Lily Little 500 blu-ray player Clovis
Mitch Mayo 450 blender Sun City
Ned Norris 400 planter Eustis
Ora Osborne 475 juicer Decatur City


In [69]:
from z3 import Solver, Int, Distinct, Implies, And, Or, sat

class Z3GridSolver:
    def __init__(self, categories):
        self._solver = Solver()
        self._cats   = categories
        self._vars   = {}
        # create one Int var per category/item
        for cat, items in categories.items():
            for item in items:
                v = Int(f"{cat}|{item}")
                self._vars[(cat, item)] = v
            # uniqueness within category
            self._solver.add(Distinct([self._vars[(cat,i)] for i in items]))
        # all domains from 1..N
        N = max(len(items) for items in categories.values())
        for v in self._vars.values():
            self._solver.add(And(v >= 1, v <= N))

    def set_equal(self, cat1, item1, cat2, item2):
        """ item1 in cat1 equals item2 in cat2 """
        self._solver.add(self._vars[(cat1,item1)] == self._vars[(cat2,item2)])

    def set_not_equal(self, cat1, item1, cat2, item2):
        self._solver.add(self._vars[(cat1,item1)] != self._vars[(cat2,item2)])

    def set_diff(self, cat_num, item_num, cat_ref, item_ref, diff):
        """ numeric cat values differ by diff: num(item_num) = num(item_ref) + diff """
        self._solver.add(self._vars[(cat_num,item_num)] == self._vars[(cat_ref,item_ref)] + diff)

    def add_either_or(self, clauseA, clauseB):
        """ each clause is a list of (cat,item,cat2,item2,op,...) constraints """
        # for simplicity here we inline the two big And(...)s 
        self._solver.add(Or(clauseA, clauseB))

    def solve(self, primary_cat):
        if self._solver.check() != sat:
            return None
        m = self._solver.model()
        # build result rows sorted by primary_cat
        rows = []
        for item in self._cats[primary_cat]:
            v = m[self._vars[(primary_cat,item)]].as_long()
            # find linked items in other categories
            row = {primary_cat: item}
            for cat in self._cats:
                if cat == primary_cat: continue
                # find other item in cat with same index
                for it in self._cats[cat]:
                    if m[self._vars[(cat,it)]].as_long() == v:
                        row[cat] = it
                        break
            rows.append((v,row))
        return [r for _,r in sorted(rows)]

In [ ]:
('$400 | Ora Osborne | planter | Eustis\n'
 '$425 | Jack Jordan | laptop | Sun City\n'
 '$450 | Mitch Mayo | juicer | Fruitland\n'
 '$475 | Lily Little | blender | Decatur City\n'
 '$500 | Ned Norris | blu-ray player | Clovis')

In [77]:
from z3 import Solver, Int, Bool, Or, And, Not, Xor, Implies, Distinct, If, sat
from typing import Dict, List, Union

class Z3Wrapper:
    """
    Wrapper around Z3 Solver providing category-based and individual constraint operations.

    Attributes:
        solver: Z3 Solver instance.
        categories: Mapping from category names to list of item identifiers.
        vars: Mapping from variable keys ("category|item") to Z3 Int/Bool variables.
    """
    def __init__(self, categories: Dict[str, List[str]]) -> None:
        """
        Initialize the wrapper with categories but do not auto-create variables.

        Inputs:
            categories: dict mapping category name to list of item names
        """
        self.solver = Solver()
        self.categories = categories
        self.vars: Dict[str, Union[Int, Bool]] = {}

    def create_category_vars(self, category: str, dtype: str = 'int', lower: int = None, upper: int = None) -> None:
        """
        Declare variables for all items in a category with specified data type and optional bounds.
        """
        items = self.categories.get(category, [])
        for item in items:
            key = f"{category}|{item}"
            if dtype == 'bool':
                v = Bool(key)
            else:
                v = Int(key)
                minv = lower if lower is not None else 1
                maxv = upper if upper is not None else len(items)
                self.solver.add(v >= minv, v <= maxv)
            self.vars[key] = v
        if dtype != 'bool' and items:
            self.solver.add(Distinct(*[self.vars[f"{category}|{item}"] for item in items]))

    def set_category_domain(self, category: str, lower: int = None, upper: int = None) -> None:
        """
        Adjust domain bounds for all integer variables in a given category.
        """
        for item in self.categories.get(category, []):
            v = self.vars.get(f"{category}|{item}")
            if v is not None and isinstance(v, Int):
                if lower is not None: self.solver.add(v >= lower)
                if upper is not None: self.solver.add(v <= upper)

    def add_eq(self, a: str, b: Union[str, int]) -> None:
        """Constrain variable a == b."""
        left = self.vars[a]
        right = self.vars[b] if isinstance(b, str) else b
        self.solver.add(left == right)

    def add_ne(self, a: str, b: Union[str, int]) -> None:
        """Constrain variable a != b."""
        left = self.vars[a]
        right = self.vars[b] if isinstance(b, str) else b
        self.solver.add(left != right)

    def add_lt(self, a: str, b: Union[str, int]) -> None:
        """Constrain variable a < b."""
        left = self.vars[a]
        right = self.vars[b] if isinstance(b, str) else b
        self.solver.add(left < right)

    def add_le(self, a: str, b: Union[str, int]) -> None:
        """Constrain variable a <= b."""
        left = self.vars[a]
        right = self.vars[b] if isinstance(b, str) else b
        self.solver.add(left <= right)

    def add_gt(self, a: str, b: Union[str, int]) -> None:
        """Constrain variable a > b."""
        left = self.vars[a]
        right = self.vars[b] if isinstance(b, str) else b
        self.solver.add(left > right)

    def add_ge(self, a: str, b: Union[str, int]) -> None:
        """Constrain variable a >= b."""
        left = self.vars[a]
        right = self.vars[b] if isinstance(b, str) else b
        self.solver.add(left >= right)

    def add_and(self, names: List[str]) -> None:
        """Constrain logical AND over given variable keys."""
        self.solver.add(And(*[self.vars[n] for n in names]))

    def add_or(self, names: List[str]) -> None:
        """Constrain logical OR over given variable keys."""
        self.solver.add(Or(*[self.vars[n] for n in names]))

    def add_xor(self, a: str, b: str) -> None:
        """Constrain logical XOR between two variable keys."""
        self.solver.add(Xor(self.vars[a], self.vars[b]))

    def add_nand(self, names: List[str]) -> None:
        """Constrain logical NAND over given variable keys."""
        self.solver.add(Not(And(*[self.vars[n] for n in names])))

    def add_nor(self, names: List[str]) -> None:
        """Constrain logical NOR over given variable keys."""
        self.solver.add(Not(Or(*[self.vars[n] for n in names])))

    def add_implies(self, cond_key: str, cond_val: Union[str,int], then_key: str, then_val: Union[str,int]) -> None:
        """
        Constrain a single implication: if cond_key==cond_val then then_key==then_val.
        """
        lhs = self.vars[cond_key]
        rhs = self.vars[cond_val] if isinstance(cond_val, str) else cond_val
        ant = lhs == rhs
        lhs2 = self.vars[then_key]
        rhs2 = self.vars[then_val] if isinstance(then_val, str) else then_val
        cons = lhs2 == rhs2
        self.solver.add(Implies(ant, cons))

    def add_conditional(self, cond_key: str, cond_val: Union[str,int],
                         then_key: str, then_val: Union[str,int],
                         else_key: str, else_val: Union[str,int]) -> None:
        """
        Constrain: if cond_key==cond_val then then_key==then_val else else_key==else_val.
        """
        # Antecedent
        ant = self.vars[cond_key] == (self.vars[cond_val] if isinstance(cond_val, str) else cond_val)
        # Branches
        then_c = self.vars[then_key] == (self.vars[then_val] if isinstance(then_val, str) else then_val)
        else_c = self.vars[else_key] == (self.vars[else_val] if isinstance(else_val, str) else else_val)
        # Add both
        self.solver.add(Implies(ant, then_c))
        self.solver.add(Implies(Not(ant), else_c))

    def add_piecewise(self, target: str,
                      cond_key: str, cond_val: Union[str,int],
                      true_val: Union[str,int], false_val: Union[str,int]) -> None:
        """
        Constrain target == If(cond_key==cond_val, true_val, false_val).
        """
        cond = self.vars[cond_key] == (self.vars[cond_val] if isinstance(cond_val, str) else cond_val)
        tv = self.vars[true_val] if isinstance(true_val, str) else true_val
        fv = self.vars[false_val] if isinstance(false_val, str) else false_val
        self.solver.add(self.vars[target] == If(cond, tv, fv))

    def check(self) -> bool:
        """Check if current constraints are satisfiable."""
        return self.solver.check() == sat

    def model(self):
        """Get the model after satisfiability check."""
        return self.solver.model() if self.solver.check() == sat else None


In [ ]:
from z3 import Solver, Int, Bool, Or, And, Not, Xor, Implies, Distinct, If, sat
from typing import Dict, List, Union

class Z3Wrapper:
    """
    Wrapper around Z3 Solver providing category-based and individual constraint operations.

    Attributes:
        solver: Z3 Solver instance.
        categories: Mapping from category names to list of item identifiers.
        vars: Mapping from variable keys ("category|item") to Z3 Int/Bool variables.
    """
    def __init__(self, categories: Dict[str, List[str]]) -> None:
        """
        Initialize the wrapper with categories but do not auto-create variables.

        Inputs:
            categories: dict mapping category name to list of item names
        """
        self.solver = Solver()
        self.categories = categories
        self.vars: Dict[str, Union[Int, Bool]] = {}

    def create_category_vars(self, category: str, dtype: str = 'int', lower: int = None, upper: int = None) -> None:
        """
        Declare variables for all items in a category with specified data type and optional bounds.

        Inputs:
            category: Category identifier matching a key in self.categories
            dtype: 'int' for Int, 'bool' for Bool
            lower: Minimum bound for Int variables (inclusive)
            upper: Maximum bound for Int variables (inclusive)
        """
        items = self.categories.get(category, [])
        vars_list = []
        for item in items:
            key = f"{category}|{item}"
            if dtype == 'bool':
                v = Bool(key)
            else:
                v = Int(key)
                minv = lower if lower is not None else 1
                maxv = upper if upper is not None else len(items)
                self.solver.add(v >= minv, v <= maxv)
            self.vars[key] = v
            vars_list.append(v)
        if dtype != 'bool' and vars_list:
            self.solver.add(Distinct(*vars_list))

    def set_category_domain(self, category: str, lower: int = None, upper: int = None) -> None:
        """
        Adjust domain bounds for all integer variables in a given category.

        Inputs:
            category: Category identifier
            lower: Minimum allowed value (inclusive)
            upper: Maximum allowed value (inclusive)
        """
        for item in self.categories.get(category, []):
            key = f"{category}|{item}"
            v = self.vars.get(key)
            if v is not None and isinstance(v, Int):
                if lower is not None:
                    self.solver.add(v >= lower)
                if upper is not None:
                    self.solver.add(v <= upper)

    def set_category_distinct(self, category: str) -> None:
        """
        Re-assert all-different constraint on a given category's integer variables.

        Inputs:
            category: Category identifier
        """
        exprs = [self.vars[f"{category}|{item}"] for item in self.categories.get(category, [])
                 if isinstance(self.vars.get(f"{category}|{item}"), Int)]
        if exprs:
            self.solver.add(Distinct(*exprs))

    # Individual constraint methods use variable keys or integer constants
    def add_eq(self, a: str, b: Union[str, int]) -> None:
        """Constrain variable a == b."""
        left = self.vars[a]
        right = self.vars[b] if isinstance(b, str) else b
        self.solver.add(left == right)

    def add_ne(self, a: str, b: Union[str, int]) -> None:
        """Constrain variable a != b."""
        left = self.vars[a]
        right = self.vars[b] if isinstance(b, str) else b
        self.solver.add(left != right)

    def add_lt(self, a: str, b: Union[str, int]) -> None:
        """Constrain variable a < b."""
        left = self.vars[a]
        right = self.vars[b] if isinstance(b, str) else b
        self.solver.add(left < right)

    def add_le(self, a: str, b: Union[str, int]) -> None:
        """Constrain variable a <= b."""
        left = self.vars[a]
        right = self.vars[b] if isinstance(b, str) else b
        self.solver.add(left <= right)

    def add_gt(self, a: str, b: Union[str, int]) -> None:
        """Constrain variable a > b."""
        left = self.vars[a]
        right = self.vars[b] if isinstance(b, str) else b
        self.solver.add(left > right)

    def add_ge(self, a: str, b: Union[str, int]) -> None:
        """Constrain variable a >= b."""
        left = self.vars[a]
        right = self.vars[b] if isinstance(b, str) else b
        self.solver.add(left >= right)

    def add_and(self, names: List[str]) -> None:
        """Constrain logical AND over given variable keys."""
        self.solver.add(And(*[self.vars[n] for n in names]))

    def add_or(self, names: List[str]) -> None:
        """Constrain logical OR over given variable keys."""
        self.solver.add(Or(*[self.vars[n] for n in names]))

    def add_xor(self, a: str, b: str) -> None:
        """Constrain logical XOR between two variable keys."""
        self.solver.add(Xor(self.vars[a], self.vars[b]))

    def add_nand(self, names: List[str]) -> None:
        """Constrain logical NAND over given variable keys."""
        self.solver.add(Not(And(*[self.vars[n] for n in names])))

    def add_nor(self, names: List[str]) -> None:
        """Constrain logical NOR over given variable keys."""
        self.solver.add(Not(Or(*[self.vars[n] for n in names])))

    def add_implies(self, a: str, b: str) -> None:
        """Constrain implication a -> b over variable keys."""
        self.solver.add(Implies(self.vars[a], self.vars[b]))

    def add_distinct(self, names: List[str]) -> None:
        """Constrain given variable keys to all be distinct."""
        self.solver.add(Distinct(*[self.vars[n] for n in names]))

    def check(self) -> bool:
        """Check if current constraints are satisfiable."""
        return self.solver.check() == sat

    def model(self):
        """Get the model after satisfiability check."""
        return self.solver.model() if self.solver.check() == sat else None


In [ ]:
Position | Attendee | Gift | Town | Price
1 | Ned Norris | blu-ray player | Clovis | $500
2 | Ora Osborne | planter | Eustis | $400
3 | Jack Jordan | blender | Fruitland | $450
4 | Mitch Mayo | juicer | Decatur City | $425
5 | Lily Little | laptop | Sun City | $475

[attendees|Ora Osborne = 2,
 gifts|blu-ray player = 1,
 attendees|Mitch Mayo = 4,
 prices|$500 = 1,
 prices|$425 = 4,
 gifts|laptop = 5,
 towns|Clovis = 1,
 gifts|planter = 2,
 attendees|Ned Norris = 1,
 gifts|blender = 3,
 towns|Eustis = 2,
 attendees|Lily Little = 5,
 towns|Fruitland = 3,
 towns|Sun City = 5,
 prices|$450 = 3,
 prices|$475 = 5,
 towns|Decatur City = 4,
 prices|$400 = 2,
 gifts|juicer = 4,
 attendees|Jack Jordan = 3]


In [ ]:
('$400 | Ora Osborne | planter | Eustis\n'
 '$425 | Jack Jordan | laptop | Sun City\n'
 '$450 | Mitch Mayo | juicer | Fruitland\n'
 '$475 | Lily Little | blender | Decatur City\n'
 '$500 | Ned Norris | blu-ray player | Clovis')

In [81]:
from z3 import *
import pandas as pd
from tabulate import tabulate

# Core Variable Functions
def create_variables(categories):
    """
    Create integer variables for each item in all categories.
    
    Args:
        categories: Dictionary with category names as keys and lists of items as values
    
    Returns:
        Dictionary mapping each item to its Z3 integer variable
    """
    variables = {}
    for category, items in categories.items():
        for item in items:
            # Create a unique variable name by combining category and item
            var_name = f"{category}_{item}"
            variables[item] = Int(var_name)
    return variables

def create_indexed_variables(primary_category, secondary_category, categories):
    """
    Create indexed variables for assignments between two categories.
    Example: Which month (index) is assigned to each flier.
    
    Args:
        primary_category: Name of the primary category (e.g., "fliers")
        secondary_category: Name of the secondary category (e.g., "months")
        categories: Dictionary with category names as keys and lists of items as values
    
    Returns:
        List of Z3 variables representing the indices in secondary_category assigned to each item in primary_category
    """
    primary_items = categories[primary_category]
    return [Int(f"{secondary_category}_for_{item}") for item in primary_items]

# Helper Functions
def get_index_lookup_functions(categories):
    """
    Create helper functions to look up indices of items in each category.
    
    Args:
        categories: Dictionary with category names as keys and lists of items as values
    
    Returns:
        Dictionary of functions to look up indices in each category
    """
    lookup_functions = {}
    
    for category, items in categories.items():
        # Create a closure to capture the current items list
        def make_lookup_function(category_items):
            def lookup_function(item_name):
                return category_items.index(item_name)
            return lookup_function
        
        lookup_functions[category] = make_lookup_function(items)
    
    return lookup_functions

# Constraint Functions
def set_domain_limit(solver, variables, lower_bound, upper_bound):
    """
    Set domain limits for all variables.
    
    Args:
        solver: Z3 Solver instance
        variables: Dictionary or list of variables
        lower_bound: Minimum value for variables
        upper_bound: Maximum value for variables
    """
    if isinstance(variables, dict):
        for var in variables.values():
            solver.add(var >= lower_bound)
            solver.add(var <= upper_bound)
    else:  # Assume it's a list
        for var in variables:
            solver.add(var >= lower_bound)
            solver.add(var <= upper_bound)

def add_distinct_constraint(solver, variables_subset):
    """
    Add a constraint that all variables in the subset must have distinct values.
    
    Args:
        solver: Z3 Solver instance
        variables_subset: List of Z3 variables that should have distinct values
    """
    solver.add(Distinct(variables_subset))

def add_equality_constraint(solver, var1, var2):
    """
    Add a constraint that two variables must be equal.
    
    Args:
        solver: Z3 Solver instance
        var1: First Z3 variable
        var2: Second Z3 variable
    """
    solver.add(var1 == var2)

def add_equality_with_offset_constraint(solver, var1, var2, offset):
    """
    Add a constraint that var1 equals var2 plus some offset.
    Example: Christie leaves 1 month after Eloise.
    
    Args:
        solver: Z3 Solver instance
        var1: First Z3 variable
        var2: Second Z3 variable
        offset: Integer offset between var1 and var2
    """
    solver.add(var1 == var2 + offset)

def add_inequality_constraint(solver, var1, var2):
    """
    Add a constraint that two variables must not be equal.
    
    Args:
        solver: Z3 Solver instance
        var1: First Z3 variable
        var2: Second Z3 variable
    """
    solver.add(var1 != var2)

def add_less_than_constraint(solver, var1, var2):
    """
    Add a constraint that var1 must be less than var2.
    
    Args:
        solver: Z3 Solver instance
        var1: First Z3 variable
        var2: Second Z3 variable
    """
    solver.add(var1 < var2)

def add_less_than_or_equal_constraint(solver, var1, var2):
    """
    Add a constraint that var1 must be less than or equal to var2.
    
    Args:
        solver: Z3 Solver instance
        var1: First Z3 variable
        var2: Second Z3 variable
    """
    solver.add(var1 <= var2)

def add_greater_than_constraint(solver, var1, var2):
    """
    Add a constraint that var1 must be greater than var2.
    
    Args:
        solver: Z3 Solver instance
        var1: First Z3 variable
        var2: Second Z3 variable
    """
    solver.add(var1 > var2)

def add_greater_than_or_equal_constraint(solver, var1, var2):
    """
    Add a constraint that var1 must be greater than or equal to var2.
    
    Args:
        solver: Z3 Solver instance
        var1: First Z3 variable
        var2: Second Z3 variable
    """
    solver.add(var1 >= var2)

def add_value_constraint(solver, var, value):
    """
    Add a constraint that a variable must equal a specific value.
    
    Args:
        solver: Z3 Solver instance
        var: Z3 variable
        value: Integer value
    """
    solver.add(var == value)

def add_not_equal_value_constraint(solver, var, value):
    """
    Add a constraint that a variable must not equal a specific value.
    
    Args:
        solver: Z3 Solver instance
        var: Z3 variable
        value: Integer value
    """
    solver.add(var != value)

def add_sum_constraint(solver, variables, total):
    """
    Add a constraint that the sum of variables equals a total.
    
    Args:
        solver: Z3 Solver instance
        variables: List of Z3 variables
        total: Sum value
    """
    solver.add(Sum(variables) == total)

# Logical operations

def add_or_constraint(solver, constraints):
    """
    Add a logical OR constraint between multiple conditions.
    
    Args:
        solver: Z3 Solver instance
        constraints: List of Z3 expressions representing conditions
    """
    solver.add(Or(constraints))

def add_and_constraint(solver, constraints):
    """
    Add a logical AND constraint between multiple conditions.
    
    Args:
        solver: Z3 Solver instance
        constraints: List of Z3 expressions representing conditions
    """
    solver.add(And(constraints))

def add_xor_constraint(solver, constraint1, constraint2):
    """
    Add a logical XOR constraint between two conditions.
    
    Args:
        solver: Z3 Solver instance
        constraint1: First Z3 expression
        constraint2: Second Z3 expression
    """
    solver.add(Xor(constraint1, constraint2))

def add_not_constraint(solver, constraint):
    """
    Add a logical NOT constraint.
    
    Args:
        solver: Z3 Solver instance
        constraint: Z3 expression to negate
    """
    solver.add(Not(constraint))

def add_nand_constraint(solver, constraint1, constraint2):
    """
    Add a logical NAND constraint between two conditions.
    
    Args:
        solver: Z3 Solver instance
        constraint1: First Z3 expression
        constraint2: Second Z3 expression
    """
    solver.add(Not(And(constraint1, constraint2)))

def add_nor_constraint(solver, constraint1, constraint2):
    """
    Add a logical NOR constraint between two conditions.
    
    Args:
        solver: Z3 Solver instance
        constraint1: First Z3 expression
        constraint2: Second Z3 expression
    """
    solver.add(Not(Or(constraint1, constraint2)))

def add_implication_constraint(solver, premise, conclusion):
    """
    Add a logical implication constraint (If premise, then conclusion).
    
    Args:
        solver: Z3 Solver instance
        premise: Z3 expression representing the premise condition
        conclusion: Z3 expression representing the conclusion condition
    """
    solver.add(Implies(premise, conclusion))

def add_if_and_only_if_constraint(solver, condition1, condition2):
    """
    Add a bi-directional implication (iff) constraint.
    
    Args:
        solver: Z3 Solver instance
        condition1: First Z3 expression
        condition2: Second Z3 expression
    """
    solver.add(condition1 == condition2)

def add_complex_case_constraint(solver, case_descriptions):
    """
    Add a constraint with multiple complex cases (OR of ANDs pattern).
    
    Args:
        solver: Z3 Solver instance
        case_descriptions: List of lists of Z3 expressions representing cases
                          Each inner list is a set of conditions that should be AND-ed together.
                          The outer list represents different cases that should be OR-ed.
    """
    cases = [And(conditions) for conditions in case_descriptions]
    solver.add(Or(cases))

def add_conditional_index_constraint(solver, index_var, index_value, condition):
    """
    Add a constraint that if index_var equals index_value, then condition must be true.
    Example: If a flier travels in February, they must have the lucky hat.
    
    Args:
        solver: Z3 Solver instance
        index_var: Z3 variable representing an index
        index_value: Integer index value
        condition: Z3 expression that must be true when index_var == index_value
    """
    solver.add(Implies(index_var == index_value, condition))

def add_cross_constraints(solver, var_list1, var_list2, constraint_func):
    """
    Add constraints between all pairs of variables from two lists.
    
    Args:
        solver: Z3 Solver instance
        var_list1: First list of Z3 variables
        var_list2: Second list of Z3 variables
        constraint_func: Function that takes two variables and returns a constraint
    """
    for var1 in var_list1:
        for var2 in var_list2:
            solver.add(constraint_func(var1, var2))

# Complex constraints

def add_if_then_else_constraint(solver, condition, then_var, else_var, result_var):
    """
    Add a constraint for if-then-else logic: if condition then result=then_var else result=else_var.
    
    Args:
        solver: Z3 Solver instance
        condition: Z3 expression for the condition
        then_var: Z3 variable for the 'then' case
        else_var: Z3 variable for the 'else' case
        result_var: Z3 variable to hold the result
    """
    solver.add(result_var == If(condition, then_var, else_var))

def add_conditional_value_constraint(solver, condition, var, value):
    """
    Add a constraint that if a condition is true, a variable must have a specific value.
    
    Args:
        solver: Z3 Solver instance
        condition: Z3 expression for the condition
        var: Z3 variable
        value: Value to set if condition is true
    """
    solver.add(Implies(condition, var == value))

def add_at_most_constraint(solver, variables, value, count):
    """
    Add a constraint that at most 'count' variables can have a specific value.
    
    Args:
        solver: Z3 Solver instance
        variables: List of Z3 variables
        value: The value being constrained
        count: Maximum number of variables that can have the value
    """
    # Create Boolean variables indicating if each variable equals the value
    indicators = [var == value for var in variables]
    
    # Sum of indicators must be <= count
    solver.add(PbLe([(indicator, 1) for indicator in indicators], count))

def add_at_least_constraint(solver, variables, value, count):
    """
    Add a constraint that at least 'count' variables must have a specific value.
    
    Args:
        solver: Z3 Solver instance
        variables: List of Z3 variables
        value: The value being constrained
        count: Minimum number of variables that must have the value
    """
    # Create Boolean variables indicating if each variable equals the value
    indicators = [var == value for var in variables]
    
    # Sum of indicators must be >= count
    solver.add(PbGe([(indicator, 1) for indicator in indicators], count))

def add_exactly_constraint(solver, variables, value, count):
    """
    Add a constraint that exactly 'count' variables must have a specific value.
    
    Args:
        solver: Z3 Solver instance
        variables: List of Z3 variables
        value: The value being constrained
        count: Exact number of variables that must have the value
    """
    # Create Boolean variables indicating if each variable equals the value
    indicators = [var == value for var in variables]
    
    # Sum of indicators must be = count
    solver.add(PbEq([(indicator, 1) for indicator in indicators], count))

def add_all_different_from_value_constraint(solver, variables, value):
    """
    Add a constraint that all variables must be different from a specific value.
    
    Args:
        solver: Z3 Solver instance
        variables: List of Z3 variables
        value: Value that all variables must differ from
    """
    for var in variables:
        solver.add(var != value)

def add_ordered_constraint(solver, variables):
    """
    Add a constraint that variables must be in ascending order.
    
    Args:
        solver: Z3 Solver instance
        variables: List of Z3 variables that should be in ascending order
    """
    for i in range(len(variables) - 1):
        solver.add(variables[i] < variables[i + 1])

def add_all_equal_constraint(solver, variables):
    """
    Add a constraint that all variables must be equal to each other.
    
    Args:
        solver: Z3 Solver instance
        variables: List of Z3 variables that should all be equal
    """
    if variables:
        for i in range(1, len(variables)):
            solver.add(variables[0] == variables[i])

# Solution and display functions

def solve_model(solver):
    """
    Check if the model is satisfiable and return the model if it is.
    
    Args:
        solver: Z3 Solver instance
    
    Returns:
        The model if satisfiable, None otherwise
    """
    if solver.check() == sat:
        return solver.model()
    else:
        print("No solution found. The constraints may be contradictory.")
        return None

def extract_solution_from_model(model, variables):
    """
    Extract variable assignments from a model.
    
    Args:
        model: Z3 model
        variables: Dictionary mapping names to Z3 variables, or list of variables
    
    Returns:
        Dictionary with variable assignments if solution exists
    """
    if model is None:
        return None
    
    if isinstance(variables, dict):
        return {name: model.evaluate(var).as_long() for name, var in variables.items()}
    else:  # Assume it's a list
        return [model.evaluate(var).as_long() for var in variables]

def resolve_indexed_solution(indexed_solutions, categories, primary_category, secondary_category):
    """
    Convert indexed solutions to a mapping between items.
    
    Args:
        indexed_solutions: List of index assignments
        categories: Dictionary with category names as keys and lists of items as values
        primary_category: Name of the primary category
        secondary_category: Name of the secondary category
    
    Returns:
        Dictionary mapping primary items to secondary items
    """
    primary_items = categories[primary_category]
    secondary_items = categories[secondary_category]
    
    resolved = {}
    for i, index in enumerate(indexed_solutions):
        primary_item = primary_items[i]
        secondary_item = secondary_items[index]
        resolved[primary_item] = secondary_item
    
    return resolved

def print_solution(solution, variables=None):
    """
    Print the solution in a simple format.
    
    Args:
        solution: Dictionary or list of variable assignments
        variables: Optional dictionary mapping names to Z3 variables (for list solutions)
    """
    if solution is None:
        print("No solution to print.")
        return
    
    print("Solution found:")
    if isinstance(solution, dict):
        for name, value in solution.items():
            print(f"{name} = {value}")
    else:  # Assume it's a list
        if variables is not None and isinstance(variables, dict):
            var_names = list(variables.keys())
            for i, value in enumerate(solution):
                if i < len(var_names):
                    print(f"{var_names[i]} = {value}")
                else:
                    print(f"Variable_{i} = {value}")
        else:
            for i, value in enumerate(solution):
                print(f"Variable_{i} = {value}")

def create_solution_table(categories, solution, var_dict=None):
    """
    Create a formatted table from the solution, organized by categories.
    
    Args:
        categories: Dictionary with category names as keys and lists of items as values
        solution: Dictionary mapping variable names to values, or list of values
        var_dict: Optional dictionary mapping variable names to indices (for list solutions)
    
    Returns:
        DataFrame or formatted string table representation
    """
    if solution is None:
        return "No solution to display"
    
    # Create a DataFrame with categories as columns
    data = {}
    
    if isinstance(solution, dict):
        for category, items in categories.items():
            # Get values for each item in this category
            values = [solution.get(item) for item in items]
            data[category] = values
    else:  # Assume it's a list
        if var_dict is not None:
            # Convert list solution to dict form using var_dict
            solution_dict = {}
            for name, idx in var_dict.items():
                if idx < len(solution):
                    solution_dict[name] = solution[idx]
            
            for category, items in categories.items():
                values = [solution_dict.get(item) for item in items]
                data[category] = values
        else:
            # Can't organize by category without mapping info
            return "Cannot create category table without variable mapping information"
    
    # Create DataFrame with items as index and categories as columns
    max_items = max(len(items) for items in categories.values())
    index = []
    for i in range(max_items):
        index.append(f"Item {i+1}")
    
    df = pd.DataFrame(data, index=index[:max_items])
    
    # For each category, fill the rows with the actual item names
    for category, items in categories.items():
        for i, item in enumerate(items):
            if i < len(index):
                df.at[f"Item {i+1}", f"{category}_name"] = item
    
    # Reorder columns to alternate between name and value
    column_order = []
    for category in categories.keys():
        column_order.append(f"{category}_name")
        column_order.append(category)
    
    # Keep only columns that exist in the DataFrame
    column_order = [col for col in column_order if col in df.columns or col.replace("_name", "") in df.columns]
    
    # Ensure all necessary columns exist
    for category in categories.keys():
        name_col = f"{category}_name"
        if name_col not in df.columns:
            df[name_col] = ""
            for i, item in enumerate(categories[category]):
                if i < len(index):
                    df.at[index[i], name_col] = item
    
    # Try to reorder again after ensuring columns exist
    try:
        df = df[[col for col in column_order if col in df.columns]]
    except:
        pass  # If reordering fails, keep original order
    
    # Create a more straightforward table for better visualization
    result_table = []
    for category, items in categories.items():
        # Add a header row for each category
        result_table.append([f"--- {category.upper()} ---", "Value"])
        for item in items:
            if isinstance(solution, dict) and item in solution:
                result_table.append([item, solution[item]])
            elif var_dict is not None and item in var_dict and var_dict[item] < len(solution):
                result_table.append([item, solution[var_dict[item]]])
        # Add a blank row between categories
        result_table.append(["", ""])
    
    # Return tabulate table with better formatting
    table_str = tabulate(result_table, headers="firstrow", tablefmt="grid")
    print(table_str)
    
    return df

def create_assignment_table(assignments, categories, primary_category, secondary_category, tertiary_category=None):
    """
    Create a table for assignment problems (who got what and when).
    
    Args:
        assignments: Dictionary or list of assignments
        categories: Dictionary with category names as keys and lists of items as values
        primary_category: The main category (e.g., "fliers")
        secondary_category: The first assignment category (e.g., "months")
        tertiary_category: Optional second assignment category (e.g., "charms")
    
    Returns:
        Formatted table showing assignments
    """
    # Get the items for each category
    primary_items = categories[primary_category]
    secondary_items = categories[secondary_category]
    tertiary_items = categories.get(tertiary_category, None) if tertiary_category else None
    
    # Prepare table data
    table_data = []
    
    # Add header row
    if tertiary_category:
        header = [primary_category.capitalize(), secondary_category.capitalize(), tertiary_category.capitalize()]
    else:
        header = [primary_category.capitalize(), secondary_category.capitalize()]
    
    table_data.append(header)
    
    # Process assignments and add rows
    if isinstance(assignments, tuple) and len(assignments) == 2:
        # Tuple of two assignment lists
        secondary_assignments, tertiary_assignments = assignments
        
        # Each index i corresponds to the primary item at i
        for i, primary_item in enumerate(primary_items):
            if i < len(secondary_assignments) and i < len(tertiary_assignments):
                sec_idx = secondary_assignments[i]
                tert_idx = tertiary_assignments[i]
                
                if sec_idx < len(secondary_items) and tert_idx < len(tertiary_items):
                    secondary_item = secondary_items[sec_idx]
                    tertiary_item = tertiary_items[tert_idx]
                    table_data.append([primary_item, secondary_item, tertiary_item])
    
    elif isinstance(assignments, dict):
        # Dictionary linking items to lists of indices
        for primary_item in primary_items:
            if primary_item in assignments:
                indices = assignments[primary_item]
                
                if isinstance(indices, tuple) and len(indices) == 2:
                    # Tuple of (secondary_index, tertiary_index)
                    sec_idx, tert_idx = indices
                    
                    if sec_idx < len(secondary_items) and tert_idx < len(tertiary_items):
                        secondary_item = secondary_items[sec_idx]
                        tertiary_item = tertiary_items[tert_idx]
                        table_data.append([primary_item, secondary_item, tertiary_item])
                elif isinstance(indices, int) and indices < len(secondary_items):
                    # Single index into secondary items
                    secondary_item = secondary_items[indices]
                    table_data.append([primary_item, secondary_item])
    
    # Return formatted table
    table_str = tabulate(table_data, headers="firstrow", tablefmt="grid")
    print(table_str)
    
    return table_str

def organize_by_secondary_category(assignments, categories, primary_category, secondary_category, tertiary_category=None):
    """
    Create a table organized by secondary category (e.g., by Month).
    
    Args:
        assignments: Dictionary or list of assignments
        categories: Dictionary with category names as keys and lists of items as values
        primary_category: The main category (e.g., "fliers")
        secondary_category: The first assignment category (e.g., "months")
        tertiary_category: Optional second assignment category (e.g., "charms")
    
    Returns:
        Formatted table organized by secondary category
    """
    # Get the items for each category
    primary_items = categories[primary_category]
    secondary_items = categories[secondary_category]
    tertiary_items = categories.get(tertiary_category, None) if tertiary_category else None
    
    # Create a mapping from secondary items to primary items
    secondary_to_primary = {}
    secondary_to_tertiary = {} if tertiary_category else None
    
    if isinstance(assignments, tuple) and len(assignments) == 2:
        # Tuple of two assignment lists
        secondary_assignments, tertiary_assignments = assignments
        
        # Each index i corresponds to the primary item at i
        for i, primary_item in enumerate(primary_items):
            if i < len(secondary_assignments):
                sec_idx = secondary_assignments[i]
                
                if sec_idx < len(secondary_items):
                    secondary_item = secondary_items[sec_idx]
                    secondary_to_primary[secondary_item] = primary_item
                    
                    if tertiary_category and i < len(tertiary_assignments):
                        tert_idx = tertiary_assignments[i]
                        if tert_idx < len(tertiary_items):
                            tertiary_item = tertiary_items[tert_idx]
                            if secondary_to_tertiary is None:
                                secondary_to_tertiary = {}
                            secondary_to_tertiary[secondary_item] = tertiary_item
    
    elif isinstance(assignments, dict):
        # Dictionary linking primary items to indices
        for primary_item, indices in assignments.items():
            if isinstance(indices, tuple) and len(indices) == 2:
                # Tuple of (secondary_index, tertiary_index)
                sec_idx, tert_idx = indices
                
                if sec_idx < len(secondary_items):
                    secondary_item = secondary_items[sec_idx]
                    secondary_to_primary[secondary_item] = primary_item
                    
                    if tertiary_category and tert_idx < len(tertiary_items):
                        tertiary_item = tertiary_items[tert_idx]
                        if secondary_to_tertiary is None:
                            secondary_to_tertiary = {}
                        secondary_to_tertiary[secondary_item] = tertiary_item
            
            elif isinstance(indices, int) and indices < len(secondary_items):
                # Single index into secondary items
                secondary_item = secondary_items[indices]
                secondary_to_primary[secondary_item] = primary_item
    
    # Prepare table data
    table_data = []
    
    # Add header row
    if tertiary_category:
        header = [secondary_category.capitalize(), primary_category.capitalize(), tertiary_category.capitalize()]
    else:
        header = [secondary_category.capitalize(), primary_category.capitalize()]
    
    table_data.append(header)
    
    # Add rows in order of secondary items
    for secondary_item in secondary_items:
        if secondary_item in secondary_to_primary:
            primary_item = secondary_to_primary[secondary_item]
            
            if tertiary_category and secondary_to_tertiary and secondary_item in secondary_to_tertiary:
                tertiary_item = secondary_to_tertiary[secondary_item]
                table_data.append([secondary_item, primary_item, tertiary_item])
            else:
                table_data.append([secondary_item, primary_item])
    
    # Return formatted table
    table_str = tabulate(table_data, headers="firstrow", tablefmt="grid")
    print("\nOrganized by", secondary_category.capitalize())
    print(table_str)
    
    return table_str

In [82]:
from z3 import *
import pandas as pd
from tabulate import tabulate

# Define categories
categories = {
    "months": ["January", "February", "March", "April", "May"],
    "fliers": ["Brandi", "Christie", "Eloise", "Florence", "Jaime"],
    "lucky_charms": ["coin", "lucky hat", "shamrock", "talisman", "wishbone"]
}

# Create variables for months assigned to each flier
month_indices = create_indexed_variables("fliers", "months", categories)

# Create variables for lucky charms assigned to each flier
charm_indices = create_indexed_variables("fliers", "lucky_charms", categories)

# Create a solver
solver = Solver()

# Set domain constraints (0-4 since we have 5 items in each category)
set_domain_limit(solver, month_indices, 0, 4)
set_domain_limit(solver, charm_indices, 0, 4)

# Each flier must have a unique month
add_distinct_constraint(solver, month_indices)

# Each flier must have a unique lucky charm
add_distinct_constraint(solver, charm_indices)